# HLW Replication

This notebook documents the R code used for the estimation of the natural rate of interest, potential GDP, and its trend growth rate for the United States, Canada, the Euro Area, and the UK, presented in "Measuring the Natural Rate of Interest: International Trends and Determinants" (Holston, Laubach, Williams 2017; henceforth HLW). It also catalogues steps to download and prepare data used in the estimation. The code documented here reflects the final December 2016 version of the paper, with an estimation sample of 1961:Q1 through 2016:Q3. 

## Code Layout and Directory Structure
There is one main R file, *run.hlw.R*, which does the following:  
    1. Prepares data to be used in the HLW estimation;  
    2. Runs the three-stage HLW estimation for each economy;   
    3. Saves output.
    
**Please note that running *run.hlw.R* and running this notebook are equivalent. This notebook walks the user through *run.hlw.R* and the functions that it calls. All code is included inline in this notebook; no R files are sourced or required.** 
    
This file calls multiple R functions and files, each of which are described and included in this guide. To run the code without modification, use the following structure:  
    1. A subdirectory titled "rawData" should contain all data described below, downloaded and saved as CSV files;  
    2. An empty subdirectory titled "inputData" should be created and will populate with prepared data;  
    3. An empty subdirectory titled "output" should be created and will populate with estimates and model output.
    4. Optional: create a subdirectory titled "Rpackages" to use as the library location for downloaded packages.
    
**For the US, raw data will be downloaded automatically. For all other economies, the user must manually download the data and save it in the rawData subdirectory.**

R Version: 3.2.2.

## Raw Data
A further description of the data we require can be found in the Data Appendix to the paper. For each economy, we require data for real GDP, inflation, and the short-term nominal interest rate, as well as a procedure to compute inflation expectations to calculate the ex ante real short-term interest rate. The inflation measure is the annualized quarterly growth rate of the specified consumer price series. Interest rates are expressed on a 365-day annualized basis.

### United States
We use real GDP and core PCE data published by the Bureau of Economic Analysis. The short-term interest rate is the annualized nominal federal funds rate, available from the Board of Governors. Because the federal funds rate frequently fell below the discount rate prior to 1965, we use the Federal Reserve Bank of New York's discount rate, part of the IMF's International Financial Statistics Yearbooks (IFS), prior to 1965. All US data can be downloaded from the St. Louis Fed's Federal Reserve Economic Data (FRED) website, and will be automatically downloaded in *prepare.rstar.data.us.R.*
#### FRED Mnemonics:   
Real GDP: GDPC1  
Core PCE: PCEPILFE  
Federal Funds Rate: FEDFUNDS  
FRBNY Discount Rate: INTDSRUSM193N

### Canada
We use real GDP from the IMF's IFS. The short-term nominal interest rate is the Bank of Canada's target for the overnight rate, taken as the end-of-period value for each month and aggregated to quarterly frequency. Prior to May 2001, we use the Bank of Canada's bank rate as the short-term interest rate. We use the BoC's core consumer price index to construct our inflation series. Prior to 1984, we use CPI containing all items. With the exception of GDP, all data is from Statistics Canada.

#### Mnemonics:
Real GDP: IFS series "Gross Domestic Product, Real, Seasonally adjusted, Index"  
Core CPI: v41690926 (Table 326-0022); Source: Statistics Canada  
CPI: v41690914 (Table 326-0022); v41690973 (Table 326-0020); Source: Statistics Canada  
Bank Rate: v122530 (Table 176-0043); Source: Statistics Canada  
Target Rate: v39079 (Table 176-0048); Source: Statistics Canada

### Euro Area
All data is from the Area Wide Model (AWM), which is available from the Euro Area Business Cycle Network. We use the core price index beginning in 1988 and the total price index prior; because data availability is longer for the non-seasonally adjusted series, we use those and seasonally adjust them. The nominal short-term interest rate is the three-month rate. Since the AWM is released annually, we update each series using data from the ECB's Statistical Data Warehouse. For example, at the time of publication we used AWM data through 2015:Q4 and updated the series using ECB data for the first three quarters of 2016.

#### AWM Mnemonics:  
Real GDP: YER  
Price Index: HICP (not seasonally adjusted)  
Core Price Index: HEX  
Nominal Short-term Rate: STN

#### ECB Statistical Data Warehouse Mnemonics:  
Real GDP: MNA.Q.Y.I8.W2.S1.S1.B.B1GQ.\_Z.\_Z.\_Z.EUR.LR.N  
Core Price Index: ICP.M.U2.N.XE0000.4.INX  
Nominal Short-term Rate: FM.Q.U2.EUR.RT.MM.EURIBOR3MD\_.HSTA  

### United Kingdom
GDP data are taken from the Office of National Statistics (ONS). Our inflation measure is core CPI; prior to 1970 we use all-items CPI; both are from the OECD. The short-term interest rate is the Bank of England's Official Bank Rate.

#### Mnemonics:  
Real GDP: Series ABMI; Source: ONS  
CPI: Series "MEI Prices: Consumer prices - all items"; Source: OECD  
Core CPI: Series "MEI Prices: Consumer prices - all items non-food, non-energy"; Source: OECD  
Nominal Short-term Rate: "Official Bank Rate History"; Source: Bank of England Statistical Interactive Database

## Basic Functions used Throughout HLW Programs
In the accompanying set of code, these functions are stored in *utilities.R*.

**Function:**     *shiftQuarter*  
**Description:**  This function takes in a (year, quarter) date in time series format and a shift number, and returns the (year, quarter) date corresponding to the shift. Positive values of shift produce leads and negative values of shift produce lags. For example, entering 2014q1 with a shift of -1 would return 2013q4. Entering 2014q1 with a shift of 1 would return 2014q2. In each case, the first argument of the function must be entered as a two-element vector, where the first element corresponds to the year and the second element corresponds to the quarter. For example, 2014q1 must be entered as "c(2014, 1)".

In [ ]:
shiftQuarter <- function(original.start,shift){
# Leads (positive values of shift)
    if (shift > 0) {
        new.start = c(0,0)
        sum = original.start[2] + shift
    
        # Get the year value
        if (sum <= 4) {
            new.start[1] = original.start[1]
        }
        else {
            new.start[1] = original.start[1] + ceiling(sum/4) - 1
        }

        # Get the quarter value
        if (sum %% 4 > 0) {
            new.start[2] = sum %% 4
        }
        else {
            new.start[2] = sum %% 4 + 4
        }
    }

# Lags (negative values of shift)
    else {
        new.start = c(0,0)
        diff = original.start[2] - abs(shift)
    
        # Get the year value
        if (diff > 0) {
            new.start[1] = original.start[1]
        }
        else {
            new.start[1] = original.start[1] - (1 + floor(abs(diff)/4))
        }

        # Get the quarter value
        if (diff %% 4 > 0) {
            new.start[2] = diff %% 4
        }
        else {
            new.start[2] = diff %% 4 + 4
        }
    }
        
return(new.start)}

**Function**: *shiftMonth*  
**Description**: This function takes in a (year, month) date in time series format and a shift number, and returns the (year, month) date corresponding to the shift. Positive values of shift produce leads and negative values of shift produce lags. For example, entering 2014m1 with a shift of -1 would return 2013m12. Entering 2014m1 with a shift of 1 would return 2014m2. In each case, the first argument of the function must be entered as a two-element vector, where the first element corresponds to the year and the second element corresponds to the month. This function is analogous to shiftQuarter().

In [ ]:
shiftMonth <- function(original.start,shift){
# Leads (positive values of shift)
    if (shift > 0) {
        new.start = c(0,0)
        sum = original.start[2] + shift
    
        # Get the year value
        if (sum <= 12) {
            new.start[1] = original.start[1]
        }
        else {
            new.start[1] = original.start[1] + ceiling(sum/12) - 1
        }

        # Get the month value
        if (sum %% 12 > 0) {
            new.start[2] = sum %% 12
        }
        else {
            new.start[2] = sum %% 12 + 12
        }
    }

# Lags (negative values of shift)
    else {
        new.start = c(0,0)
        diff = original.start[2] - abs(shift)
    
        # Get the year value
        if (diff > 0) {
            new.start[1] = original.start[1]
        }
        else {
            new.start[1] = original.start[1] - (1 + floor(abs(diff)/12))
        }

        # Get the month value
        if (diff %% 12 > 0) {
            new.start[2] = diff %% 12
        }
        else {
            new.start[2] = diff %% 12 + 12
        }
    }
        
return(new.start)}

**Function**: *getFRED*  
**Description**: This function downloads data from FRED. It returns quarterly data. User must provide the FRED url.

In [ ]:
getFRED <- function(url, freq = "Quarterly") {
    # Download the data from FRED
    
    #download.file(url, destfile = 'FREDtemp.txt', method = "wget")
    #FREDraw <- readLines('FREDtemp.txt')
    
    txt.file.name <- paste0("rawData/",substr(url, regexpr('[a-zA-z0-9]*.txt',url),1000))
    if (!file.exists(txt.file.name)){
        # Download the data from FRED
        #download.file(url, destfile = 'FREDtemp.txt', method = "wget")
        system(paste0('wget --no-check-certificate "', url, '"'))
        system(paste('mv',substr(url, regexpr('[a-zA-z0-9]*.txt',url),1000),txt.file.name))
    }
    FREDraw <- readLines(txt.file.name) 

    # Frequency
    freq.FRED <- gsub(' ', '',substr(FREDraw[which(regexpr('Frequency', FREDraw)==1)],
                                     (nchar('Frequency')+2),100))    

    # Where does the data start
    datastart = which(gsub(' ', '',FREDraw)=='DATEVALUE') - 2

    #data <- read.table('FREDtemp.txt', skip = datastart, header = TRUE)
    data <- read.table(txt.file.name, skip = datastart, header = TRUE)

    first.year  <- as.numeric(format(as.Date(data$DATE[1]),'%Y'))
    first.month <- as.numeric(format(as.Date(data$DATE[1]),'%m'))
    
    # Adjust frequency
    if (freq.FRED == 'Quarterly'){
        first.q  <- (first.month-1)/3 + 1
        data.tis <- tis(data$VALUE, start = c(first.year, first.q), tif = 'quarterly')
    } else if (freq.FRED == 'Monthly') {
        data.tis <- tis(data$VALUE, start = c(first.year, first.month), tif = 'monthly')
    }

    # Convert frequency
    if (freq.FRED == 'Monthly' & freq == 'Quarterly') {
        data.tis <- convert(data.tis, tif = 'quarterly', method = 'constant', observed. = 'averaged')
    }

return(data.tis)} 

**Function**: *splice*  
**Description**: This function splices two series, with the series s2 beginning at splice.date and extended back using the growth rate at the splice.date times series s1. The freq argument accepts two values - 'quarterly' and 'monthly' - but it could be modified to take more.

In [ ]:
splice <- function(s1, s2, splice.date, freq) {
    t <- splice.date #renaming for convenience
    if (freq == "quarterly" | freq == "Quarterly") {
        t.minus.1 <- shiftQuarter(t,-1)
    }
    else if (freq == "monthly" | freq == "Monthly") {
        t.minus.1 <- shiftMonth(t,-1)
    }
    else { stop("You must enter 'quarterly' or 'monthly' for freq.") }
    ratio <- as.numeric(window(s2,start = t, end = t)/
                        window(s1,start = t, end = t))

return(mergeSeries(ratio*window(s1,end = t.minus.1),window(s2, start = t)))}

**Function**: *gradient*  
**Description**: This function computes the gradient of a function f given a vector input x.

In [ ]:
gradient <- function(f, x, delta = x * 0 + 1.0e-5) {
    g <- x * 0
    for (i in 1:length(x)) {
        x1 <- x
        x1[i] <- x1[i] + delta[i]
        f1 <- f(x1)
        x2 <- x
        x2[i] <- x2[i] - delta[i]
        f2 <- f(x2)
        g[i] <- (f1 - f2) / delta[i] / 2
    }
return(g)}

## Load Packages
Checks for necessary R packages and installs them if not found. The "tis" package is used to manage time series data. The "mFilter" packages contains the hpfilter() function. We use the "nloptr" package for optimization. The "seasonal" package is an interface to the US Census Bureau software X-13-ARIMA-SEATS. It uses X-13 binaries which are installed by the dependent "x13binary" packages; users may need to manually specify the path to the binaries.

In [ ]:
if (!require("tis")) {install.packages("tis", lib = "Rpackages"); library("tis", lib = "Rpackages")} # Time series package
if (!require("mFilter")) {install.packages("mFilter", lib = "Rpackages"); library("mFilter", lib = "Rpackages")} # HP filter
if (!require("nloptr")) {install.packages("nloptr", lib = "Rpackages"); library("nloptr", lib = "Rpackages")} # Optimization
if (!require("seasonal")) {install.packages("seasonal", lib = "Rpackages"); library('seasonal', lib = "Rpackages")} # Seasonal adjustment
Sys.setenv(X13_PATH = "X13")

## Data Preparation
We use four R scripts, one for each economy titled *prepare.rstar.data.XX.R*, to prepare the data to be used in the estimation. Inputs are the raw data described above. The files are described below. In the accompanying set of code, they are sourced in *run.hlw.R*.

### *prepare.rstar.data.us.R*
This file compiles and prepares the data used in HLW for the US.

Manually specify start and end dates for the prepared data to be used in the estimation. The variable *data.start* should be 4 quarters prior to the estimation start date.

In [ ]:
data.start <- c(1960,1)
data.end   <- c(2015,4)

Import data using the getFRED() function in *utilities.R*. For this connection to work, the user must have the *wget* command line utility. Alternatively, you can comment out the data import section of the file and manually download the data from FRED.

In [ ]:
gdp             <- getFRED(paste0('https://fred.stlouisfed.org/',
                                     'data/GDPC1.txt'))

price.index     <- getFRED(paste0('https://fred.stlouisfed.org/',                                  
                                     'data/PCEPILFE.txt'))

ny.discount     <- getFRED(paste0('https://fred.stlouisfed.org/',
                                     'data/INTDSRUSM193N.txt'))

fed.funds       <- getFRED(paste0('https://fred.stlouisfed.org/',
                                     'data/FEDFUNDS.txt'))

Prepare data:
* Take the log of real GDP.
* Create an annualized core inflation series from the price index.
* Construct a measure of inflation expectations as a four-quarter moving average of past inflation.
* Express interest rate data on a 365-day basis.
* Splice FRBNY discount rates with the federal funds rate in 1965:Q1. 

In [ ]:
# Take log of real GDP
gdp.log <- log(gdp)

# Create an annualized inflation series using the price index
inflation <- 400*log(price.index/Lag(price.index, k=1))

# Inflation expectations measure: 4-quarter moving average of past inflation
inflation.expectations <- (inflation + Lag(inflation, k=1) + Lag(inflation, k=2) + Lag(inflation, k=3))/4

# Express interest rate data on a 365-day basis
ny.discount.eff <- 100*((1+ny.discount/36000)^365 -1)
fed.funds.eff   <- 100*((1+fed.funds/36000)^365 -1)

# NY Fed discount rate is used prior to 1965; thereafter, use the effective federal funds rate
interest <- mergeSeries(window(ny.discount.eff, end = c(1964,4)),window(fed.funds.eff, start = c(1965,1)))

Output data to *inputData/rstar.data.us.csv*

In [ ]:
data.out <- window(cbind(gdp.log, inflation, inflation.expectations, interest),start = data.start, end = data.end)
write.table(data.out,file = 'inputData/rstar.data.us.csv', sep = ',',
            col.names = TRUE, quote = FALSE, na = '.', row.names = FALSE)

### *prepare.rstar.data.ca.R*  
This file compiles and prepares the data used in HLW for Canada.

Manually specify start and end dates for the prepared data to be used in the estimation. The variable *data.start* should be 4 quarters prior to the estimation start date. Manually specify the start dates for the two CPI series and the core CPI series.

In [ ]:
data.start <- c(1960,1)
data.end   <- c(2015,4)

Data described in **Raw Data** section must be downloaded by the user prior to running this file. Code is set up such that each CANSIM series is in its own CSV with time as rows.

**Import GDP Data**  
**PRIOR STEPS**:  
    1. Download data from the International Financial Statistics (IFS) Database on the IMF website:  
       Series: "Gross Domestic Product, Real, Seasonally adjusted, Index"
    2. Save data as a CSV and specify the file name in gdp.file

In [ ]:
gdp.file <- "rawData/canada_gdp_ifs.csv"
gdp.data <- read.table(gdp.file, skip = 1, header = TRUE, sep = ',', stringsAsFactors = FALSE)
gdp      <- tis(gdp.data$Canada, start = data.start, tif = 'quarterly')

**Import Bank Rate Data**  
**PRIOR STEPS:**  
    1. Download data from the Statistics Canada (CANSIM) website:  
       Series: v122530 
    2. Save data as a CSV and specify the file name in bank.rate.file

In [ ]:
bank.rate.file <- "rawData/cansim_bank_rate_v122530.csv"
bank.rate.data <- read.table(bank.rate.file, skip = 2, header = TRUE, sep = ',', stringsAsFactors = FALSE)
bank.rate.m    <- tis(bank.rate.data$v122530, start = data.start, tif='monthly')

**Import Target Rate Data**  
**PRIOR STEPS:** 
    1. Download data from the Statistics Canada (CANSIM) website:  
       Series: v39079        
    2. Save data as a CSV and specify the file name in target.rate.file
    3. Manually specify start date of downloaded target rate data.  

In [ ]:
target.rate.start <- c(2001,1)
target.rate.file  <- "rawData/cansim_target_rate_v39079.csv"
target.rate.data  <- read.table(target.rate.file, skip = 2, header = TRUE, sep = ',', stringsAsFactors = FALSE)
target.rate.data$Daily <- as.Date(target.rate.data$Daily, "%b %d %Y")

# Format data as time series with daily frequency
if (!require("xts")) {install.packages("xts"); library("xts")}
target.rate.d <- as.xts(target.rate.data$v39079, order.by = target.rate.data$Daily, frequency = 365)
# Remove data with zero values (these indicate non-business days)
target.rate.d <- target.rate.d[!(target.rate.d==0.00)]
# Aggregate data to monthly frequency by taking end-of-period values
target.rate.m <- apply.monthly(target.rate.d, tail, n=1)
detach("package:xts")
detach("package:zoo")
# Format data as a tis time series with monthly frequency
target.rate.m <- as.tis(data.frame(target.rate.m)$target.rate.m, start = target.rate.start, tif = 'monthly')

**Import Core CPI and CPI Data**  
**PRIOR STEPS:** 
    1. Download data from the Statistics Canada (CANSIM) website:
       Series: v41690926, v41690914, v41690973 (core CPI, CPI, and CPI to be used for backcasting, respectively)
    2. Save data as a CSV and specify the file name in core.cpi.file, cpi.file, cpi.back.file
    3. Manually specify start dates for each series.
   
**NOTES:**
* Series v41690914 begins in Jan. 1992 and will be used as the CPI series thereafter;
* Growth rates from series v41690973 are used to extend the CPI series back to 1959.
* Series v41690914 and v41690926 are already seasonally adjusted but v41690973 is not, so we seasonally adjust it.

In [ ]:
core.cpi.start <- c(1984,1)
cpi.start      <- c(1992,1)
cpi.back.start <- c(1959,1)

core.cpi.file  <- "rawData/cansim_core_cpi_v41690926.csv"
cpi.file       <- "rawData/cansim_cpi_v41690914.csv"
cpi.back.file  <- "rawData/cansim_cpi_back_v41690973.csv"

core.cpi.data  <- read.table(core.cpi.file, skip = 2, header = TRUE, sep = ',', stringsAsFactors = FALSE)
cpi.data       <- read.table(cpi.file, skip = 2, header = TRUE, sep = ',', stringsAsFactors = FALSE)
cpi.back.data  <- read.table(cpi.back.file, skip = 2, header = TRUE, sep = ',', stringsAsFactors = FALSE)

# Format data as time series with monthly frequency
core.cpi.m     <- tis(core.cpi.data$v41690926, start = core.cpi.start, tif = 'monthly')
cpi.m          <- tis(cpi.data$v41690914, start = cpi.start, tif = 'monthly')
cpi.back.m.nsa <- tis(cpi.back.data$v41690973, start = cpi.back.start, tif = 'monthly')

# Seasonally adjust CPI Growth Rate data (other two series are already SA)
cpi.back.m <- final(seas(as.ts(naWindow(cpi.back.m.nsa),freq=12)))
cpi.back.m <- as.tis(cpi.back.m,start=cpi.back.start,tif='monthly') 

Prepare data:
* Take the log of real GDP.
* Create an all-items price index using series v41690914 from Jan. 1992 to present. Uses growth rates of series v41690973 to extend the series back to Jan. 1959.
* Create annualized inflation and core inflation series using the spliced CPI price index and core CPI price index, respectively.
* Our inflation measure uses core inflation beginning in 1984Q2 and all-items inflation prior.
* Construct a measure of inflation expectations as a four-quarter moving average of past inflation.
* Create an interest rate series using the target rate beginning in May 2001 and the bank rate prior, and aggregates this series to quarterly frequency.
* Express interest rates on a 365-day basis.

In [ ]:
# Take log of real GDP
gdp.log <- log(gdp)

# CPI series: cpi.m is used from Jan 1992 to present and extended back to
# Jan 1959 using growth rates with splice() function in utilities.R
cpi.series.m <- splice(cpi.back.m, cpi.m, cpi.start, freq = 'monthly')

# Aggregate core CPI and CPI series to quarterly frequency by taking the average
core.cpi.q <- aggregate(core.cpi.m, nf = 4, FUN = mean)
cpi.q      <- aggregate(cpi.series.m, nf = 4, FUN = mean)

# Create annualized core inflation and inflation series using the price indices
core.inflation.q <- 400*log(core.cpi.q/Lag(core.cpi.q, k=1))
inflation.q      <- 400*log(cpi.q/Lag(cpi.q, k=1))

# Final inflation series: CPI series is used prior to 1984q2;
# thereafter, use core CPI series
inflation <- mergeSeries(window(inflation.q, end = core.cpi.start),
                         window(core.inflation.q, start = shiftQuarter(core.cpi.start,1)))

# Inflation expectations measure: 4-quarter moving average of past inflation
inflation.expectations <- (inflation + Lag(inflation, k=1) + Lag(inflation, k=2) + Lag(inflation, k=3))/4

# Bank rate is used prior to May 2001; thereafter, use the target rate
interest.m <- mergeSeries(window(bank.rate.m, end = c(2001,4)),window(target.rate.m, start = c(2001,5)))

# Aggregate interest rate data to quarterly frequency by taking the average
interest.q <- aggregate(interest.m, nf = 4, FUN = mean)

# Express interest rate data on a 365-day basis
interest <- 100*((1+interest.q/36000)^365 -1)

Output data to *inputData/rstar.data.ca.csv*

In [ ]:
data.out <- window(cbind(gdp.log, inflation, inflation.expectations, interest),start = data.start, end = data.end)
write.table(data.out,file = 'inputData/rstar.data.ca.csv', sep = ',',
            col.names = TRUE, quote = FALSE, na = '.', row.names = FALSE)

### *prepare.rstar.data.ea.R*
This file compiles and prepares the data used in HLW for the Euro Area.

Manually specify start and end dates for the prepared data to be used in the estimation. The variable *data.start* should be 4 quarters prior to the estimation start date. Note that in HLW the estimation sample begins later for the Euro Area than for other economies due to data availability. Manually specify the start date for the core CPI series (Mnemonic: HEX).

In [ ]:
data.start <- c(1971,1)
data.end   <- c(2015,4)

core.cpi.start <- c(1987,4)

The Area Wide Model data described in **Raw Data** section must be downloaded by the user prior to running this file.

**Import Area Wide Model Data**  
**PRIOR STEPS:**  
    1. Download data from the Euro Area Business Cycle Network website.
    2. Save data as a CSV and specify the file name in awm.file.  

**NOTE:** We seasonally adjust the non-seasonally-adjusted CPI and core CPI series (HICP and HEX) rather than using the provided seasonally adjusted series (HICPSA and HEXSA) because the latter series are only available for part of the sample.

In [ ]:
awm.file <- 'rawData/area_wide_model_ea.csv'
awm.data <- read.table(awm.file,header=TRUE,sep=',')[,c('DATE','YER','STN','HEX','HICP')]

awm.start    <- c(as.numeric(substr(awm.data[1,'DATE'],1,4)),
                  as.numeric(substr(awm.data[1,'DATE'],6,7)))

gdp          <- tis(awm.data$YER, start = awm.start, tif = 'quarterly')

data.end     <- end(as.ts(gdp))

cpi.nsa      <- tis(awm.data$HICP, start = awm.start, tif = 'quarterly')
core.cpi.nsa <- tis(awm.data$HEX, start = awm.start, tif = 'quarterly')
core.cpi.nsa <- window(core.cpi.nsa, start = core.cpi.start)
interest.q   <- tis(awm.data$STN, start = awm.start, tif = 'quarterly')

**Import ECB Data (for extending series past AWM end date)**  
**PRIOR STEPS:** 
    1. Download data from the ECB's Statistical Data Warehouse (see Sec. 3.3 for mnemonics).
    2. Save data as CSV files and specify names in variable.file.
    3. Ensure data is sorted with dates ascending.
    4. Specify start dates and splice dates where applicable.
    5. Splice GDP and interest series.

In [ ]:
gdp.file      <- 'rawData/ea_gdp.csv'
gdp.data      <- read.table(gdp.file,header=FALSE,sep=',',skip=5)
gdp.start.ecb <- c(1995,1)

gdp.ecb       <- tis(gdp.data$V2, start = gdp.start.ecb, tif = 'quarterly')

gdp.spliced   <- splice(gdp, gdp.ecb, gdp.start.ecb, "Quarterly")

core.cpi.file <- 'rawData/ea_hicp.csv'
core.cpi.data <- read.table(core.cpi.file,header=FALSE,sep=',',skip=5)
core.cpi.start.ecb   <- c(1996,1)
core.cpi.splice.date <- c(2016,1)

core.cpi.nsa.m.ecb <- tis(core.cpi.data$V2, start = core.cpi.start.ecb, tif = 'monthly')

interest.file <- 'rawData/ea_nominal_rate.csv'
interest.data <- read.table(interest.file,header=FALSE,sep=',',skip=5)
interest.start.ecb   <- c(1994,1)
interest.splice.date <- c(2016,1)

interest.q.ecb <- tis(interest.data$V2, start = interest.start.ecb, tif = 'quarterly')

interest.spliced <- mergeSeries(window(interest.q, end = shiftQuarter(interest.splice.date,-1)),
                                window(interest.q.ecb, start = interest.splice.date))

Prepare data:
* Take the log of real GDP.
* Aggregate ECB core CPI data to quarterly from monthly frequency.
* Splice ECB core CPI data with AWM core CPI data.
* Create annualized inflation and core inflation series using the CPI price index and core CPI price index, respectively.
* Our inflation measure uses core inflation beginning in 1988Q1 and all-items inflation prior.
* Construct a measure of inflation expectations as a four-quarter moving average of past inflation.
* Express interest rates on a 365-day basis.

In [ ]:
# Take log of real GDP
gdp.log <- log(gdp.spliced)

# Aggregate ECB core CPI data to quarterly from monthly frequency
core.cpi.nsa.ecb <- convert(core.cpi.nsa.m.ecb, tif = 'quarterly', observed = 'averaged')

# Splice ECB core CPI data with AWM core CPI data in 2015q4
core.cpi.nsa.spliced <- splice(core.cpi.nsa, core.cpi.nsa.ecb, shiftQuarter(core.cpi.splice.date, -1), "Quarterly")

# Seasonally adjust CPI and core CPI data and re-format as tis series
cpi <- final(seas(as.ts(naWindow(cpi.nsa),freq=4)))
cpi <- as.tis(cpi,start=awm.start,tif='quarterly')

core.cpi <- final(seas(as.ts(naWindow(core.cpi.nsa.spliced),freq=4)))
core.cpi <- as.tis(core.cpi,start=core.cpi.start,tif='quarterly')

# Create annualized core inflation and inflation series using the price indices
core.inflation.q <- 400*log(core.cpi/Lag(core.cpi, k=1))
inflation.q      <- 400*log(cpi/Lag(cpi, k=1))

# Final inflation series: CPI series is used prior to 1988q1;
# thereafter, use core CPI series
inflation <- mergeSeries(window(inflation.q, end = core.cpi.start),window(core.inflation.q, start = shiftQuarter(core.cpi.start,1)))

# Inflation expectations measure: 4-quarter moving average of past inflation
inflation.expectations <- (inflation + Lag(inflation, k=1) + Lag(inflation, k=2) + Lag(inflation, k=3))/4

# Express interest rate data on a 365-day basis
interest <- 100*((1+interest.spliced/36000)^365 -1)

Output data to *inputData/rstar.data.ea.csv*

In [ ]:
data.out <- window(cbind(gdp.log, inflation, inflation.expectations, interest),start = data.start, end = data.end)
write.table(data.out,file = 'inputData/rstar.data.ea.csv', sep = ',',
            col.names = TRUE, quote = FALSE, na = '.', row.names = FALSE)

### *prepare.rstar.data.uk.R*
This file compiles and prepares the data used in HLW for the UK.

Manually specify start and end dates for the prepared data to be used in the estimation. The variable *data.start* should be 4 quarters prior to the estimation start date. Manually specify the start date for the core CPI series.

In [ ]:
data.start <- c(1960,1)
data.end   <- c(2015,4)

Data described in **Raw Data** section must be downloaded by the user prior to running this file.

** Import GDP Data**  
**PRIOR STEPS:**  
    1. Download data from the ONS website:
       Series: ABMI: "Gross Domestic Product: chained volume measures: Seasonally adjusted (Millions of pounds)
    2. Save data as a CSV and specify the file name in gdp.file

In [ ]:
gdp.file  <- "rawData/uk_gdp_ons_abmi.csv"
gdp.data  <- read.table(gdp.file, skip = 7, header = FALSE, sep = ',', stringsAsFactors = FALSE) 
gdp       <- tis(gdp.data$V2, start = data.start, tif='quarterly')

**Import Core CPI and CPI Data**  
**PRIOR STEPS:** 
    1. Download data from the OECD website:
       Source: Consumer Prices (MEI); Series: Consumer prices - all items; 
       Consumer prices - all items non-food, non-energy
    2. Save both data series in one CSV and specify the file name in cpi.file.
    3. Manually specify start dates for each series.

In [ ]:
core.cpi.start <- c(1970,1)
cpi.start      <- c(1959,1)

cpi.file     <- "rawData/uk_price_indices.csv"
cpi.data     <- read.table(cpi.file, skip = 0, header = TRUE, sep = ',', stringsAsFactors = FALSE)
core.cpi.nsa <- tis(cpi.data$core_cpi, start = cpi.start, tif = 'quarterly')
core.cpi.nsa <- window(core.cpi.nsa, start = core.cpi.start)
cpi.nsa      <- tis(cpi.data$cpi, start = cpi.start, tif = 'quarterly')

**Import Bank Rate Data**  
**PRIOR STEPS:**
    1. Download data from the BOE website:
       Bank of England Statistical Interactive Database - official Bank Rate history
    2. Manually edit the historical since 1694 series 
       1. Include only 1960 to present, but enter 11/20/1958 change as 1/1/1960 so series has starting value
       2. Populate rows with years. Remove sub-headers and rows of spaces
          In Excel, select area and choose F5, "Special", "Blanks", Delete
       3. Code assume the setup has 4 columns: "year","day","month","rate"

In [ ]:
bank.rate.file      <- "rawData/uk_boe_bank_rate_changes.csv"
bank.rate.data      <- read.table(bank.rate.file, skip = 0, header = TRUE, sep = ',', stringsAsFactors = FALSE)
bank.rate.data$date <- as.Date(paste(bank.rate.data$month,bank.rate.data$day,bank.rate.data$year), "%b %d %Y")
bank.rate.data      <- subset(bank.rate.data, select = c("date","rate"))

Convert series of changes in bank rate to a daily rate series:
* Variable bank.rate.data contains changes in the BOE bank rate (dates are included only if a change occurs).
* Variable bank.rate.changes.d is a daily time series with NA values inserted for dates on which a change does not occur.
* Variable bank.rate.d is a daily series created by carrying forward the last value when a value is NA using the na.locf() function in the 'xts' package.

In [ ]:
date.seq            <- seq(from = bank.rate.data$date[1], to = as.Date(ti(data.end,tif='quarterly')), by = '1 day')
if (!require("xts")) {install.packages("xts"); library("xts")} # Time series library
bank.rate.changes.d <- data.frame(bank.rate = with(bank.rate.data, rate[match(date.seq, date)]))
bank.rate.d         <- as.xts(bank.rate.changes.d, order.by = date.seq, frequency = 365)
bank.rate.d         <- na.locf(bank.rate.d)
detach("package:xts") # Remove packages to avoid masking base functions
detach("package:zoo")
bank.rate.d         <- tis(bank.rate.d$bank.rate, end = ti(data.end, tif='quarterly'), tif = 'daily')[,1]

Prepare data:
* Take the log of real GDP.
* Seasonally adjust the CPI and core CPI series.
* Create annualized inflation and core inflation series using the CPI price index and core CPI price index, respectively.
* Our inflation measure uses core inflation beginning in 1970Q2 and all-items inflation prior.
* Construct a measure of inflation expectations as a four-quarter moving average of past inflation.
* Aggregate the bank rate series to quarterly frequency and express interest rates on a 365-day basis. 

In [ ]:
# Take log of real GDP
gdp.log <- log(gdp)

# Seasonally adjust CPI and core CPI data and re-format as tis series
cpi <- final(seas(as.ts(naWindow(cpi.nsa),freq=4)))
cpi <- as.tis(cpi,start=cpi.start,tif='quarterly')

core.cpi <- final(seas(as.ts(naWindow(core.cpi.nsa),freq=4)))
core.cpi <- as.tis(core.cpi,start=core.cpi.start,tif='quarterly')

# Create annualized core inflation and inflation series using the price indices
core.inflation.q <- 400*log(core.cpi/Lag(core.cpi, k=1))
inflation.q      <- 400*log(cpi/Lag(cpi, k=1))

# Final inflation series: CPI series is used prior to 1970q2;
# thereafter, use core CPI series
inflation <- mergeSeries(window(inflation.q, end = core.cpi.start),
                         window(core.inflation.q, start = shiftQuarter(core.cpi.start,1)))

# Inflation expectations measure: 4-quarter moving average of past inflation
inflation.expectations <- (inflation + Lag(inflation, k=1) + Lag(inflation, k=2) + Lag(inflation, k=3))/4

# Aggregate bank rate data to quarterly frequency by taking the average
interest.q <- convert(bank.rate.d, tif = 'quarterly', observed = 'averaged')

# Express interest rate data on a 365-day basis
interest <- 100*((1+interest.q/36000)^365 -1)

Output data to *inputData/rstar.data.uk.csv*

In [ ]:
data.out <- window(cbind(gdp.log, inflation, inflation.expectations, interest),start = data.start, end = data.end)
write.table(data.out,file = 'inputData/rstar.data.uk.csv', sep = ',',
            col.names = TRUE, quote = FALSE, na = '.', row.names = FALSE)

## Define Variables 
In this section, the sample period, constraints, and variables to be used throughout the estimation are defined.

In [ ]:
# Upper bound on a_3 parameter (slope of the IS curve)
a3.constraint <- -0.0025

# Lower bound on b_2 parameter (slope of the Phillips curve)
b2.constraint <- 0.025

# Set the start and end dates of the estimation sample (format is c(year,quarter))
sample.start <- c(1961,1)
sample.end   <- c(2015,4)

# Set the estimation sample start date for the Euro Area 
ea.sample.start <- c(1972,1)

# The estimation process uses data beginning 4 quarters prior to the sample start
data.start    <- shiftQuarter(sample.start,-4)
ea.data.start <- shiftQuarter(ea.sample.start,-4)

# Set start index for y
g.pot.start.index <- 1 + ti(shiftQuarter(sample.start,-3),'quarterly')-ti(data.start,'quarterly')

# Set column names for CSV output
output.col.names <- c("Date","rstar","g","z","output gap","",
                      "All results are output from the Stage 3 model.",rep("",8),
                      "Standard Errors","Date","y*","r*","g","","rrgap")

# Set number of iterations for Monte Carlo standard error procedure
niter <- 5000

# Because the MC standard error procedure is time consuming, we include a run switch
# Set run.se to TRUE to run the procedure
run.se <- TRUE

# Removes the seed setting if one exists
rm(.Random.seed)

## Estimation
The results reported in Holston, Laubach, and Williams (2017) are based on our estimation method described in Section 2.2 of the paper. The estimation proceeds in sequential steps through three stages, each of which is implemented in an R program. These programs, as well as programs to obtain median unbiased estimates of the signal-to-noise ratios, apply the Kalman filter, and estimate parameters using maximum likelihood are described and included below.

### Main Estimation Program: *run.hlw.estimation.R*
The function *run.hlw.estimation.R* is called by *run.hlw.R* once for each economy. It takes as inputs the key variables for the given economy: log output, inflation, and the real and nominal short-term interest rates, as well as the specified constraints on $a_r$ and $b_y$. It calls the programs *rstar.stage1.R*, *rstar.stage2.R*, and *rstar.stage3.R* to run the three stages of the HLW estimation and returns output for each stage. Additionally, it calls the programs *median.unbiased.estimator.stage1.R* and *median.unbiased.estimator.stage2.R* to obtain the signal-to-noise ratios $\lambda_g$ and $\lambda_z$ and returns their values.

The programs *unpack.parameters.stage1.R*, *unpack.parameters.stage2.R*, and *unpack.parameters.stage3.R* set up coefficient matrices for the corresponding state-space models for the given parameter vectors.

In all stages, we impose the constraint $b_y \geq 0.025$. In stages 2 and 3, we impose $a_r \leq -0.0025$. 

In [ ]:
run.hlw.estimation <- function(log.output, inflation, real.interest.rate, nominal.interest.rate,
                               a3.constraint = NA, b2.constraint = NA, run.se = TRUE) {
    # Running the stage 1 model
    out.stage1 <- rstar.stage1(log.output,
                               inflation,
                               b2.constraint)

    # Median unbiased estimate of lambda_g
    lambda.g <- median.unbiased.estimator.stage1(out.stage1$potential.smoothed)

    # Running the stage 2 model
    out.stage2 <- rstar.stage2(log.output,
                               inflation,
                               real.interest.rate,
                               lambda.g,
                               a3.constraint,
                               b2.constraint)
    
    # Median unbiased estimate of lambda_z
    lambda.z <- median.unbiased.estimator.stage2(out.stage2$y, out.stage2$x)

    # Running the stage 3 model
    out.stage3 <- rstar.stage3(log.output,
                               inflation,
                               real.interest.rate,
                               nominal.interest.rate,
                               lambda.g,
                               lambda.z,
                               a3.constraint,
                               b2.constraint,
                               run.se)

    return(list(out.stage1=out.stage1,out.stage2=out.stage2,out.stage3=out.stage3,
                lambda.g=lambda.g,lambda.z=lambda.z))
}

### The Stage 1, 2, and 3 State-Space Models
This sections presents the state-space models, and the next section documents the corresponding R programs. Notation matches that of Hamilton (1994) and is also used in the R programs. All of the state-space models can be cast in the form

\begin{equation}
\mathbf{y}_t = \mathbf{A}' \cdot \mathbf{x}_t + \mathbf{H}' \cdot \xi_t + \mathbf{v}_t
\end{equation}

\begin{equation}
\xi_t = \mathbf{F} \cdot \xi_{t-1} + \epsilon_t
\end{equation}

Here, $\mathbf{y}_t$ is a vector of contemporaneous endogenous variables, while $\mathbf{x}_t$ is a vector of exogenous and lagged exogenous variables. $\xi_t$ is vector of unobserved states. The vectors of stochastic disturbances $\mathbf{v}_t$ and $\epsilon_t$ are assumed to be Gaussian and mututally uncorrelated, with mean zero and covariance matrices $\mathbf{R}$ and $\mathbf{Q}$, respectively. The covariance matrix $\mathbf{R}$ is always assumed to be diagonal.

For each model, there is a corresponding vector of parameters to be estimated by maximum likelihood. Because maximum likelihood estimates of the innovations to $g$ and $z$, $\sigma_g$ and $\sigma_z$, are likely to be biased towards zero (see Section 2.2 of HLW for explanation), we use Stock and Watson's (1998) mediun unbiased estimator to obtain estimates of two ratios, $\lambda_g \equiv \frac{\sigma_g}{\sigma_{y^*}}$ and $\lambda_z \equiv \frac{a_r \sigma_z}{\sigma_{\tilde{y}}}$. We impose these ratios when estimating the remaining model parameters by maximum likelihood.

### The Stage 1 Model
The first-stage model, which corresponds to the *rstar.stage1.R* program, can be represented by the following matrices: 

$$ \mathbf{y}_t = \left[y_t, \pi_t\right]' $$

$$ \mathbf{x}_t = \left[y_{t-1},y_{t-2},\pi_{t-1},\pi_{t-2,4}\right]' $$

$$ \mathbf{\xi}_t = \left[y_t^*,y_{t-1}^*,y_{t-2}^*\right]' $$


$$ \mathbf{H}' = \begin{bmatrix} 1 & -a_{y,1} & -a_{y,2} \\ 0 & -b_y & 0 \end{bmatrix} $$

$$ \mathbf{A}' = \begin{bmatrix} a_{y,1} & a_{y,2} & 0 & 0 \\ b_y & 0 & b_\pi & 1-b_\pi \end{bmatrix} $$

$$ \mathbf{F} = \begin{bmatrix} 1 & 0 & 0 \\ 1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix} $$

$$ \mathbf{Q} = \begin{bmatrix} \sigma_{y*}^2 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \end{bmatrix} $$


The vector of parameters to be estimated by maximum likelihood is as follows:

$$\theta_1=\left[a_{y,1},a_{y,2},b_\pi,b_y,g,\sigma_{\widetilde{y}},\sigma_\pi,\sigma_{y*}\right]$$  

### *unpack.parameters.stage1.R*
This function generates the above matrices given $\theta_1$ as input.

In [ ]:
unpack.parameters.stage1 <- function(parameters, y.data, x.data, xi.00, P.00) {
  A         <- matrix(0, 4, 2)
  A[1:2, 1] <- parameters[1:2]  # a_y,1, a_y,2
  A[1, 2]   <- parameters[4]    # b_y
  A[3, 2]   <- parameters[3]    # b_pi
  A[4, 2]   <- 1-parameters[3]  # 1 - b_pi
  
  H         <- matrix(0, 3, 2)  
  H[1, 1]   <- 1
  H[2:3, 1] <- -parameters[1:2] # -a_y,1, -a_y,2
  H[2, 2]   <- -parameters[4]   # -b_y

  R         <- diag(c(parameters[6]^2, parameters[7]^2)) # sigma_y~, sigma_pi
  Q         <- matrix(0, 3, 3)
  Q[1, 1]   <- parameters[8]^2  # sigma_y*

  F <- matrix(0, 3, 3)
  F[1, 1] <- F[2, 1] <- F[3, 2] <- 1

  # Make the data stationary
  y.data[, 1] <- y.data[, 1] - 1:dim(y.data)[1] * parameters[5]
  x.data[, 1] <- x.data[, 1] - 0:(dim(x.data)[1]-1) * parameters[5]
  x.data[, 2] <- x.data[, 2] - -1:(dim(x.data)[1]-2) * parameters[5]
      
return(list("xi.00"=xi.00, "P.00"=P.00, "F"=F, "Q"=Q, "A"=A, "H"=H, "R"=R, "x.data"=x.data, "y.data"=y.data))}

### The Stage 2 Model
The second-stage model, which corresponds to the *rstar.stage2.R* program, can be represented by the following matrices:

$$ \mathbf{y}_t = \left[y_{t},\pi_{t}\right]' $$

$$ \mathbf{x}_t = \left[y_{t-1},y_{t-2},r_{t-1},r_{t-2},\pi_{t-1},\pi_{t-2,4},1\right]' $$ 

$$ \mathbf{\xi}_t = \left[y_t^*,y_{t-1}^*,y_{t-2}^*,g_{t-1}\right]' $$

$$ \mathbf{H}' = \begin{bmatrix} 1 & -a_{y,1} & -a_{y,2} & a_g \\ 0 & -b_y & 0 & 0 \end{bmatrix} $$

$$ \mathbf{A}' = \begin{bmatrix} a_{y,1} & a_{y,2} & \frac{a_r}{2} & \frac{a_r}{2} & 0 & 0 & a_0 \\ b_y & 0 & 0 & 0 & b_\pi & 1-b_\pi & 0 \end{bmatrix} $$

$$ \mathbf{F} = \begin{bmatrix} 1 & 0 & 0 & 1 \\ 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} $$ 

$$ \mathbf{Q} = \begin{bmatrix} \sigma_{y*}^2 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & \left(\lambda_g\sigma_{y*}\right)^2 \end{bmatrix} $$


The vector of parameters to be estimated by maximum likelihood is as follows: 

$$ \theta_2=\left[a_{y,1},a_{y,2},a_r,a_0,a_g,b_\pi,b_y,\sigma_{\widetilde{y}},\sigma_\pi,\sigma_{y*}\right] $$  

### *unpack.parameters.stage2.R*
This function generates the above matrices given $\theta_2$ as input.

In [ ]:
unpack.parameters.stage2 <- function(parameters, y.data, x.data, lambda.g, xi.00=NA, P.00=NA) {
  A         <- matrix(0, 2, 7)
  A[1, 1:2] <- parameters[1:2]   # a_y,1, a_y,2
  A[1, 3:4] <- parameters[3]/2   # a_r/2
  A[1, 7]   <- parameters[4]     # a_0
  A[2, 1]   <- parameters[7]     # b_y
  A[2, 5]   <- parameters[6]     # b_pi
  A[2, 6]   <- 1 - parameters[6] # 1 - b_pi
  A         <- t(A)
  
  H         <- matrix(0, 2, 4)
  H[1, 1  ] <- 1
  H[1, 2:3] <- -parameters[1:2] # -a_y,1, -a_y,2
  H[1, 4  ] <- parameters[5]    # a_g
  H[2, 2]   <- -parameters[7]   # -b_y
  H         <- t(H)

  R         <- diag(c(parameters[8]^2, parameters[9]^2)) # sigma_y~, sigma_pi
  Q         <- matrix(0, 4, 4)
  Q[1, 1]   <- parameters[10]^2              # sigma_y*
  Q[4, 4]   <- (lambda.g * parameters[10])^2 # sigma_y* 

  F <- matrix(0, 4, 4)
  F[1, 1] <- F[1, 4] <- F[2, 1] <- F[3, 2] <- F[4,4] <- 1
  
return(list("xi.00"=xi.00, "P.00"=P.00, "F"=F, "Q"=Q, "A"=A, "H"=H, "R"=R, "x.data"=x.data, "y.data"=y.data))}

### The Stage 3 Model
The third-stage model, which corresponds to the *rstar.stage3.R* program, can be represented by the following matrices:

$$ \mathbf{y}_t = \left[y_{t},\pi_{t}\right]' $$

$$ \mathbf{x}_t = \left[y_{t-1},y_{t-2},r_{t-1},r_{t-2},\pi_{t-1},\pi_{t-2,4}\right]' $$

$$ \mathbf{\xi}_t = \left[y_t^*,y_{t-1}^*,y_{t-2}^*,g_{t-1},g_{t-2},z_{t-1},z_{t-2}\right]' $$

$$ \mathbf{H}' = \begin{bmatrix} 1 & -a_{y,1} & -a_{y,2} & \frac{-a_r}{2} & \frac{-a_r}{2} & \frac{-a_r}{2} & \frac{-a_r}{2} \\ 0 & -b_y & 0 & 0 & 0 & 0 & 0 \end{bmatrix} $$

$$ \mathbf{A}' = \begin{bmatrix} a_1 & a_2 & \frac{a_r}{2} & \frac{a_r}{2} & 0 & 0 \\ b_y & 0 & 0 & 0 & b_\pi & 1-b_\pi \end{bmatrix} $$

$$ \mathbf{F} = \begin{bmatrix} 1 & 0 & 0 & 1 & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 & 0 \end{bmatrix} $$

$$ \mathbf{Q} = \begin{bmatrix} \left(1+\lambda_g^2\right)\sigma_{y*}^2 & 0 & 0 & \left(\lambda_g\sigma_{y*}\right)^2 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ \left(\lambda_g\sigma_{y*}\right)^2 & 0 & 0 & \left(\lambda_g\sigma_{y*}\right)^2 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & \left(\frac{\lambda_z\sigma_{\widetilde{y}}}{a_r}\right)^2 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 \end{bmatrix} $$

The vector of parameters to be estimated by maximum likelihood is as follows:
$$\theta_3=\left[a_{y,1},a_{y,2},a_r,b_\pi,b_y,\sigma_{\widetilde{y}},\sigma_\pi,\sigma_{y*}\right]$$  

### *unpack.parameters.stage3.R*
This function generates the coefficient matrices above given $\theta_3$ as input.

In [ ]:
unpack.parameters.stage3 <- function(parameters, y.data, x.data, lambda.g, lambda.z, xi.00, P.00) {
  A         <- matrix(0, 2, 6)   
  A[1, 1:2] <- parameters[1:2]   # a_y,1, a_y,2
  A[1, 3:4] <- parameters[3]/2   # a_r/2
  A[2, 1]   <- parameters[5]     # b_y
  A[2, 5]   <- parameters[4]     # b_pi
  A[2, 6]   <- 1 - parameters[4] # 1 - b_pi
  A         <- t(A)

  
  H         <- matrix(0, 2, 7)
  H[1, 1]   <- 1
  H[1, 2:3] <- -parameters[1:2]       # a_y,1, a_y,2
  H[1, 4:5] <- -parameters[3] * 2     # -a_r/2 (annualized)
  H[1, 6:7] <- -parameters[3]/2       # -a_r/2
  H[2, 2]   <- -parameters[5]         # -b_y
  H         <- t(H)

  R         <- diag(c(parameters[6]^2, parameters[7]^2)) # sigma_y~, sigma_pi

  Q         <- matrix(0, 7, 7)
  Q[1, 1]   <- (1+lambda.g^2)*parameters[8]^2                  # sigma_y*
  Q[1, 4]   <- Q[4, 1] <- Q[4, 4]<- (lambda.g*parameters[8])^2 # sigma_y*
  Q[6, 6]   <- (lambda.z*parameters[6]/parameters[3])^2        # sigma_y~/a_r

  F <- matrix(0, 7, 7)
  F[1, 1] <- F[1, 4] <- F[2, 1] <- F[3, 2] <- F[4,4] <- F[5,4]<- F[6,6] <- F[7,6] <- 1
 
return(list("xi.00"=xi.00, "P.00"=P.00, "F"=F, "Q"=Q, "A"=A, "H"=H, "R"=R, "x.data"=x.data, "y.data"=y.data))}

### R Programs to Run the State-Space Models

### *rstar.stage1.R*
This function runs the model in the first stage of the HLW estimation.

In [ ]:
rstar.stage1 <- function(log.output,
                         inflation,
                         b2.constraint=NA) {

  stage <- 1
  
  # Data must start 4 quarters before the estimation period
  T <- length(log.output) - 4

  # Original output gap estimate
  x.og <- cbind(rep(1,T+4), 1:(T+4))
  y.og <- log.output
  output.gap <- (y.og - x.og %*% solve(t(x.og) %*% x.og, t(x.og) %*% y.og)) * 100

  # Initialization of state vector for Kalman filter using HP trend of log output
  log.output.hp.trend <- hpfilter(log.output,freq=36000,type="lambda",drift=FALSE)$trend
  g.pot <- log.output.hp.trend[(g.pot.start.index):length(log.output.hp.trend)]
  xi.00 <- c(100*g.pot[3:1])

  # IS curve
  y.is <- output.gap[5:(T+4)]
  x.is <- cbind(output.gap[4:(T+3)], output.gap[3:(T+2)])
  b.is <- solve(t(x.is) %*% x.is, t(x.is) %*% y.is)
  r.is <- y.is - x.is %*% b.is
  s.is <- sqrt(sum(r.is^2) / (length(r.is)-(dim(x.is)[2])))

  # Phillips curve
  y.ph <- inflation[5:(T+4)]
  x.ph <- cbind(inflation[4:(T+3)],
                (inflation[3:(T+2)]+inflation[2:(T+1)]+inflation[1:T])/3,
                output.gap[4:(T+3)])                
  b.ph <- solve(t(x.ph) %*% x.ph, t(x.ph) %*% y.ph)
  r.ph <- y.ph - x.ph %*% b.ph
  s.ph <- sqrt(sum(r.ph^2) / (length(r.ph)-(dim(x.ph)[2])))
  
  y.data <- cbind(100 * log.output[5:(T+4)],
                  inflation[5:(T+4)])
  x.data <- cbind(100 * log.output[4:(T+3)],
                  100 * log.output[3:(T+2)],
                  inflation[4:(T+3)],
                  (inflation[3:(T+2)]+inflation[2:(T+1)]+inflation[1:T])/3)

  # Starting values for the parameter vector
  initial.parameters <- c(b.is, b.ph[1], b.ph[3], 0.85, s.is, s.ph, 0.5)

  # Set an upper and lower bound on the parameter vectors:
  # The vector is unbounded unless values are otherwise specified
  theta.lb <- c(rep(-Inf,length(initial.parameters)))
  theta.ub <- c(rep(Inf,length(initial.parameters)))
  
  # Set a lower bound for the Phillips curve slope (b_2) of b2.constraint, if not NA
  # In HLW, b2.constraint = 0.025
  if (!is.na(b2.constraint)) {
      if (initial.parameters[4] < b2.constraint) {
          initial.parameters[4] <- b2.constraint
      }
      theta.lb[4] <- b2.constraint
  }

  # Set the initial covariance matrix (see footnote 6) 
  P.00 <- calculate.covariance(initial.parameters, theta.lb, theta.ub, y.data, x.data, stage, NA, NA, xi.00)
  
  # Get parameter estimates via maximum likelihood
  f <- function(theta) {return(-log.likelihood.wrapper(theta, y.data, x.data, stage, NA, NA, xi.00, P.00)$ll.cum)}
  nloptr.out <- nloptr(initial.parameters, f, eval_grad_f=function(x) {gradient(f, x)},
                       lb=theta.lb,ub=theta.ub,
                       opts=list("algorithm"="NLOPT_LD_LBFGS","xtol_rel"=1.0e-8))
  theta <- nloptr.out$solution

  log.likelihood <- log.likelihood.wrapper(theta, y.data, x.data, stage, NA, NA, xi.00, P.00)$ll.cum

  # Get state vectors (xi.tt, xi.ttm1, xi.tT, P.tt, P.ttm1, P.tT) via Kalman filter
  states <- kalman.states.wrapper(theta, y.data, x.data, stage, NA, NA, xi.00, P.00)

  # One-sided (filtered) estimates  
  potential.filtered  <- states$filtered$xi.tt[,1]/100
  output.gap.filtered <- y.data[,1] - (potential.filtered * 100)
  
  # Two-sided (smoothed) estimates
  potential.smoothed  <- as.vector(states$smoothed$xi.tT[,1])/100
  output.gap.smoothed <- y.data[,1] - (potential.smoothed * 100)

  # Save variables to return
  return.list                <- list()
  return.list$theta          <- theta
  return.list$log.likelihood <- log.likelihood
  return.list$states         <- states
  return.list$xi.00          <- xi.00
  return.list$P.00           <- P.00
  return.list$potential.filtered  <- potential.filtered
  return.list$output.gap.filtered <- output.gap.filtered
  return.list$potential.smoothed  <- potential.smoothed
  return.list$output.gap.smoothed <- output.gap.smoothed
  return(return.list)
}

### *rstar.stage2.R*
This function runs the model in the second stage of the HLW estimation.

In [ ]:
rstar.stage2 <- function(log.output,
                         inflation,
                         real.interest.rate,
                         lambda.g,
                         a3.constraint=NA,
                         b2.constraint=NA) {
                         
  stage <- 2

  # Data must start 4 quarters before the estimation period  
  T <- length(log.output) - 4

  # Original output gap estimate
  x.og <- cbind(rep(1,T+4), 1:(T+4))
  y.og <- log.output
  output.gap <- (y.og - x.og %*% solve(t(x.og) %*% x.og, t(x.og) %*% y.og)) * 100

  # Initialization of state vector for Kalman filter using HP trend of log output
  log.output.hp.trend <- hpfilter(log.output,freq=36000,type="lambda",drift=FALSE)$trend
  g.pot <- log.output.hp.trend[(g.pot.start.index):length(log.output.hp.trend)]
  g.pot.diff <- diff(g.pot)
  xi.00 <- c(100*g.pot[3:1],100*g.pot.diff[2])

  # IS curve
  y.is <- output.gap[5:(T+4)]
  x.is <- cbind(output.gap[4:(T+3)], output.gap[3:(T+2)],
                (real.interest.rate[4:(T+3)] + real.interest.rate[3:(T+2)])/2,
                rep(1,T))
  b.is <- solve(t(x.is) %*% x.is, t(x.is) %*% y.is)
  r.is <- as.vector(y.is - x.is %*% b.is)
  s.is <- sqrt(sum(r.is^2) / (length(r.is)-(dim(x.is)[2])))

  # Phillips curve
  y.ph <- inflation[5:(T+4)]
  x.ph <- cbind(inflation[4:(T+3)],
                (inflation[3:(T+2)]+inflation[2:(T+1)]+inflation[1:T])/3,
                output.gap[4:(T+3)])
  b.ph <- solve(t(x.ph) %*% x.ph, t(x.ph) %*% y.ph)
  r.ph <- y.ph - x.ph %*% b.ph
  s.ph <- sqrt(sum(r.ph^2) / (length(r.ph)-(dim(x.ph)[2])))
  
  y.data <- cbind(100 * log.output[5:(T+4)],
                  inflation[5:(T+4)])
  x.data <- cbind(100 * log.output[4:(T+3)],
                  100 * log.output[3:(T+2)],
                  real.interest.rate[4:(T+3)],
                  real.interest.rate[3:(T+2)],                  
                  inflation[4:(T+3)],
                  (inflation[3:(T+2)]+inflation[2:(T+1)]+inflation[1:T])/3,
                  rep(1,T))

  # Starting values for the parameter vector
  initial.parameters <- c(b.is, -b.is[3], b.ph[1], b.ph[3], s.is, s.ph, 0.5)

  # Set an upper and lower bound on the parameter vectors:
  # The vector is unbounded unless values are otherwise specified
  theta.lb <- c(rep(-Inf,length(initial.parameters)))
  theta.ub <- c(rep(Inf,length(initial.parameters)))

  # Set a lower bound for the Phillips curve slope (b_2) of b2.constraint, if not NA
  # In HLW, b2.constraint = 0.025
  if (!is.na(b2.constraint)) {
      if (initial.parameters[7] < b2.constraint) {
          initial.parameters[7] <- b2.constraint
      }
      theta.lb[7] <- b2.constraint
  }

  # Set an upper bound for the IS curve slope (a_3) of a3.constraint, if not NA
  # In HLW, a3.constraint = -0.0025
  if (!is.na(a3.constraint)) {
      if (initial.parameters[3] > a3.constraint) {
          initial.parameters[3] <- a3.constraint
      }
      theta.ub[3] <- a3.constraint      
  }

  # Set the initial covariance matrix (see footnote 6)
  P.00 <- calculate.covariance(initial.parameters, theta.lb, theta.ub, y.data, x.data, stage, lambda.g, NA, xi.00)
  
  # Get parameter estimates via maximum likelihood
  f <- function(theta) {return(-log.likelihood.wrapper(theta, y.data, x.data, stage, lambda.g, NA, xi.00, P.00)$ll.cum)}
  nloptr.out <- nloptr(initial.parameters, f, eval_grad_f=function(x) {gradient(f, x)},
                       lb=theta.lb,ub=theta.ub,
                       opts=list("algorithm"="NLOPT_LD_LBFGS","xtol_rel"=1.0e-8))
  theta <- nloptr.out$solution

  log.likelihood <- log.likelihood.wrapper(theta, y.data, x.data, stage, lambda.g, NA, xi.00, P.00)$ll.cum
  
  # Get state vectors (xi.tt, xi.ttm1, xi.tT, P.tt, P.ttm1, P.tT) via Kalman filter
  states <- kalman.states.wrapper(theta, y.data, x.data, stage, lambda.g, NA, xi.00, P.00)

  # Two-sided (smoothed) estimates
  trend.smoothed      <- states$smoothed$xi.tt[,4] * 4
  potential.smoothed  <- c(states$smoothed$xi.tT[1, 3:2], states$smoothed$xi.tT[,1])
  output.gap.smoothed <- 100 * log.output[3:(T+4)] - potential.smoothed

  # Inputs for median.unbiased.estimator.stage2.R
  y <- output.gap.smoothed[3:length(output.gap.smoothed)]
  x <- cbind(output.gap.smoothed[2:(length(output.gap.smoothed)-1)],
             output.gap.smoothed[1:(length(output.gap.smoothed)-2)],
             (x.data[,3]+x.data[,4])/2,
             states$smoothed$xi.tT[,4],
             rep(1,T))

  # One-sided (filtered) estimates  
  trend.filtered      <- states$filtered$xi.tt[,4] * 4
  potential.filtered  <- states$filtered$xi.tt[,1]/100
  output.gap.filtered <- y.data[,1] - (potential.filtered * 100)
  
  # Save variables to return
  return.list <- list()
  return.list$y              <- y
  return.list$x              <- x
  return.list$theta          <- theta
  return.list$log.likelihood <- log.likelihood
  return.list$states         <- states
  return.list$xi.00          <- xi.00
  return.list$P.00           <- P.00
  return.list$trend.filtered      <- trend.filtered
  return.list$potential.filtered  <- potential.filtered
  return.list$output.gap.filtered <- output.gap.filtered
  return.list$trend.smoothed      <- trend.smoothed
  return.list$potential.smoothed  <- potential.smoothed
  return.list$output.gap.smoothed <- output.gap.smoothed
  return(return.list)
}

### *rstar.stage3.R*
This function runs the model in the third stage of the HLW estimation.

In [ ]:
rstar.stage3 <- function(log.output,
                         inflation,
                         real.interest.rate,
                         nominal.interest.rate,
                         lambda.g,
                         lambda.z,
                         a3.constraint=NA,
                         b2.constraint=NA,
                         run.se = TRUE) {

  stage <- 3
  
  # Data must start 4 quarters before the estimation period
  T <- length(log.output) - 4

  # Original output gap estimate
  x.og <- cbind(rep(1,T+4), 1:(T+4))
  y.og <- log.output
  output.gap <- (y.og - x.og %*% solve(t(x.og) %*% x.og, t(x.og) %*% y.og)) * 100

  # Initialization of state vector for Kalman filter using HP trend of log output
  log.output.hp.trend <- hpfilter(log.output,freq=36000,type="lambda",drift=FALSE)$trend
  g.pot <- log.output.hp.trend[(g.pot.start.index):length(log.output.hp.trend)]
  g.pot.diff <- diff(g.pot)
  xi.00 <- c(100*g.pot[3:1],100*g.pot.diff[2:1],0,0)

  # IS curve
  y.is <- output.gap[5:(T+4)]
  x.is <- cbind(output.gap[4:(T+3)], output.gap[3:(T+2)],
                (real.interest.rate[4:(T+3)] + real.interest.rate[3:(T+2)])/2,
                rep(1,T))
  b.is <- solve(t(x.is) %*% x.is, t(x.is) %*% y.is)
  r.is <- as.vector(y.is - x.is %*% b.is)
  s.is <- sqrt(sum(r.is^2) / (length(r.is)-(dim(x.is)[2])))

  # Phillips curve
  y.ph <- inflation[5:(T+4)]
  x.ph <- cbind(inflation[4:(T+3)],
                (inflation[3:(T+2)]+inflation[2:(T+1)]+inflation[1:T])/3,
                output.gap[4:(T+3)])
  b.ph <- solve(t(x.ph) %*% x.ph, t(x.ph) %*% y.ph)
  r.ph <- y.ph - x.ph %*% b.ph
  s.ph <- sqrt(sum(r.ph^2) / (length(r.ph)-(dim(x.ph)[2])))
  
  y.data <- cbind(100 * log.output[5:(T+4)],
                  inflation[5:(T+4)])
  x.data <- cbind(100 * log.output[4:(T+3)],
                  100 * log.output[3:(T+2)],
                  real.interest.rate[4:(T+3)],
                  real.interest.rate[3:(T+2)],                  
                  inflation[4:(T+3)],
                  (inflation[3:(T+2)]+inflation[2:(T+1)]+inflation[1:T])/3)

  # Starting values for the parameter vector  
  initial.parameters <- c(b.is[1:3], b.ph[1], b.ph[3], s.is, s.ph, 0.7) 

  # Set an upper and lower bound on the parameter vectors:
  # The vector is unbounded unless values are otherwise specified  
  theta.lb <- c(rep(-Inf,length(initial.parameters)))
  theta.ub <- c(rep(Inf,length(initial.parameters)))

  # Set a lower bound for the Phillips curve slope (b_2) of b2.constraint, if not NA
  # In HLW, b2.constraint = 0.025  
  if (!is.na(b2.constraint)) {
      print(paste0("Setting a lower bound on b_2 of ",as.character(b2.constraint)))
      if (initial.parameters[5] < b2.constraint) {
          initial.parameters[5] <- b2.constraint
      }
      theta.lb[5] <- b2.constraint
  }

  # Set an upper bound for the IS curve slope (a_3) of a3.constraint, if not NA
  # In HLW, a3.constraint = -0.0025  
  if (!is.na(a3.constraint)) {
      print(paste0("Setting an upper bound on a_3 of ",as.character(a3.constraint)))
      if (initial.parameters[3] > a3.constraint) {
          initial.parameters[3] <- a3.constraint
      }
      theta.ub[3] <- a3.constraint      
  }

  # Set the initial covariance matrix (see footnote 6)   
  P.00 <- calculate.covariance(initial.parameters, theta.lb, theta.ub, y.data, x.data, stage, lambda.g, lambda.z, xi.00)
  
  # Get parameter estimates via maximum likelihood
  f <- function(theta) {return(-log.likelihood.wrapper(theta, y.data, x.data, stage, lambda.g, lambda.z, xi.00, P.00)$ll.cum)}
  nloptr.out <- nloptr(initial.parameters, f, eval_grad_f=function(x) {gradient(f, x)},
                       lb=theta.lb,ub=theta.ub,
                       opts=list("algorithm"="NLOPT_LD_LBFGS","xtol_rel"=1.0e-8))
  theta <- nloptr.out$solution
  
  log.likelihood <- log.likelihood.wrapper(theta, y.data, x.data, stage, lambda.g, lambda.z, xi.00, P.00)$ll.cum

  # Get state vectors (xi.tt, xi.ttm1, xi.tT, P.tt, P.ttm1, P.tT) via Kalman filter
  states <- kalman.states.wrapper(theta, y.data, x.data, stage, lambda.g, lambda.z, xi.00, P.00)

  # If run.se = TRUE, compute standard errors for estimates of the states (see footnote 7) and report run time
  if (run.se) {
      ptm <- proc.time()
      se <- kalman.standard.errors(T, states, theta, y.data, x.data, stage, lambda.g, lambda.z, xi.00, P.00, 
                                   niter, a3.constraint, b2.constraint)
      print("Standard error procedure run time")
      print(proc.time() - ptm)
  }
  
  # One-sided (filtered) estimates
  trend.filtered      <- states$filtered$xi.tt[,4] * 4
  z.filtered          <- states$filtered$xi.tt[,6]
  rstar.filtered      <- trend.filtered + z.filtered
  potential.filtered  <- states$filtered$xi.tt[,1]/100
  output.gap.filtered <- y.data[,1] - (potential.filtered * 100)

  # Two-sided (smoothed) estimates
  trend.smoothed      <- states$smoothed$xi.tt[,4] * 4
  z.smoothed          <- states$smoothed$xi.tt[,6]
  rstar.smoothed      <- trend.smoothed + z.smoothed
  potential.smoothed  <- states$smoothed$xi.tt[,1]/100
  output.gap.smoothed <- y.data[,1] - (potential.smoothed * 100)
   
  # Save variables to return
  return.list <- list()
  return.list$rstar.filtered      <- rstar.filtered
  return.list$trend.filtered      <- trend.filtered
  return.list$z.filtered          <- z.filtered
  return.list$potential.filtered  <- potential.filtered
  return.list$output.gap.filtered <- output.gap.filtered
  return.list$rstar.smoothed      <- rstar.smoothed
  return.list$trend.smoothed      <- trend.smoothed
  return.list$z.smoothed          <- z.smoothed
  return.list$potential.smoothed  <- potential.smoothed
  return.list$output.gap.smoothed <- output.gap.smoothed
  return.list$theta               <- theta
  return.list$log.likelihood      <- log.likelihood
  return.list$states              <- states
  return.list$xi.00               <- xi.00
  return.list$P.00                <- P.00
  return.list$y.data              <- y.data
  return.list$initial.parameters  <- initial.parameters
  if (run.se) { return.list$se    <- se }
  return(return.list)
}

### R Programs for Median Unbiased Estimators

### *median.unbiased.estimator.stage1.R*
This function computes the exponential Wald statistic of Andrews and Ploberger (1994) for a structural break with unknown break date from the first difference of the preliminary estimate of the natural rate of output from the stage 1 model to obtain the median unbiased estimate of $\lambda_g$.

In [ ]:
median.unbiased.estimator.stage1 <- function(series) {
    T <- length(series)
    y <- 400 * diff(series)

    stat <- rep(T-2*4)
    for (i in 4:(T-5)) {
      xr <- cbind(rep(1, T-1), c(rep(0,i),rep(1,T-i-1)))
      xi <- solve(t(xr) %*% xr)
      b  <- solve(t(xr) %*% xr, t(xr) %*% y)
      s3 <- sum((y-xr%*%b)^2)/(T-2-1)
      stat[i+1-4] = b[2]/sqrt(s3*xi[2,2])
    }

    ew <- 0
    for (i in 1:length(stat)) {
        ew <- ew+exp(stat[i]^2/2)
    }
    ew  <- log(ew/length(stat))
    mw  <- sum(stat^2) / length(stat)
    qlr <- max(stat^2)

    # Values are from Table 3 in Stock and Watson (1998)
    # Test Statistic: Exponential Wald (EW)
    valew <- c(0.426, 0.476, 0.516, 0.661, 0.826, 1.111,
               1.419, 1.762, 2.355, 2.91,  3.413, 3.868, 4.925,
               5.684, 6.670, 7.690, 8.477, 9.191, 10.693, 12.024,
               13.089, 14.440, 16.191, 17.332, 18.699, 20.464,
               21.667, 23.851, 25.538, 26.762, 27.874)
    # Test Statistic: Mean Wald (MW)
    valmw <- c(0.689, 0.757, 0.806, 1.015, 1.234, 1.632,
               2.018, 2.390, 3.081, 3.699, 4.222, 4.776, 5.767,
               6.586, 7.703, 8.683, 9.467, 10.101, 11.639, 13.039,
               13.900, 15.214, 16.806, 18.330, 19.020, 20.562,
               21.837, 24.350, 26.248, 27.089, 27.758)
    # Test Statistic: QLR
    valql <- c(3.198, 3.416, 3.594, 4.106, 4.848, 5.689,
               6.682, 7.626, 9.16,  10.66, 11.841, 13.098, 15.451,
               17.094, 19.423, 21.682, 23.342, 24.920, 28.174, 30.736,
               33.313, 36.109, 39.673, 41.955, 45.056, 48.647, 50.983,
               55.514, 59.278, 61.311, 64.016)
    
    lame <- NA
    lamm <- NA
    lamq <- NA

    # Median-unbiased estimator of lambda_g for given values of the test
    # statistics are obtained using the procedure described in the 
    # footnote to Stock and Watson (1998) Table 3.
    if (ew <= valew[1]) {
        lame <- 0
    } else {
        for (i in 1:(length(valew)-1)) {
            if ((ew > valew[i]) & (ew <= valew[i+1])) {
                lame <- i-1+(ew-valew[i])/(valew[i+1]-valew[i])
            }
        }
    }
    if (mw <= valmw[1]) {
        lamm <- 0
    } else {
        for (i in 1:(length(valmw)-1)) {
            if ((mw > valmw[i]) & (mw <= valmw[i+1])) {
                lamm <- i-1+(mw-valmw[i])/(valmw[i+1]-valmw[i])
            }
        }
    }
    if (qlr <= valql[1]) {
        lamq <- 0
    } else {
        for (i in 1:(length(valql)-1)) {
            if ((qlr > valql[i]) & (qlr <= valql[i+1])) {
                lamq <- i-1+(qlr-valql[i])/(valql[i+1]-valql[i])
            }
        }
    }
    if (is.na(lame) | is.na(lamm) | is.na(lamq)) {
        print("At least one statistic has an NA value. 
              Check to see if your EW, MW, and/or QLR value is outside of Table 3.")
    }
    
    stats <- c(ew, mw, qlr)
    lams  <- c(lame, lamm, lamq)
    return(lame/(T-1))
}

### *median.unbiased.estimator.stage2.R*
This function applies the exponential Wald test for an intercept shift in the IS equation at an unknown date to obtain the median unbiased estimate of $\lambda_z$, taking as input estimates from the stage 2 model.

In [ ]:
median.unbiased.estimator.stage2 <- function(y, x) {
    T <- dim(x)[1]
    stat <- rep(0, T-2*4+1)
    for (i in 4:(T-4)) {
        xr <- cbind(x, c(rep(0, i), rep(1, T-i)))
        xi <- solve(t(xr)%*%xr)
        b <- solve(t(xr)%*%xr,t(xr)%*%y)
        s3 <- sum((y-xr%*%b)^2)/(T-dim(xr)[2])
        stat[i+1-4] <- b[dim(xr)[2]]/sqrt(s3*xi[dim(xr)[2],dim(xr)[2]])
    }
    ew <- 0
    for (i in 1:length(stat)) {
        ew <- ew+exp((stat[i]^2)/2)
    }
    ew <- log(ew/length(stat))
    mw <- mean(stat^2)
    qlr <- max(stat^2)

    # Values are from Table 3 in Stock and Watson (1998)
    # Test Statistic: Exponential Wald (EW)
    valew <- c(0.426, 0.476, 0.516, 0.661, 0.826, 1.111,
               1.419, 1.762, 2.355, 2.91,  3.413, 3.868, 4.925,
               5.684, 6.670, 7.690, 8.477, 9.191, 10.693, 12.024,
               13.089, 14.440, 16.191, 17.332, 18.699, 20.464,
               21.667, 23.851, 25.538, 26.762, 27.874)
    # Test Statistic: Mean Wald (MW)
    valmw <- c(0.689, 0.757, 0.806, 1.015, 1.234, 1.632,
               2.018, 2.390, 3.081, 3.699, 4.222, 4.776, 5.767,
               6.586, 7.703, 8.683, 9.467, 10.101, 11.639, 13.039,
               13.900, 15.214, 16.806, 18.330, 19.020, 20.562,
               21.837, 24.350, 26.248, 27.089, 27.758)
    # Test Statistic: QLR
    valql <- c(3.198, 3.416, 3.594, 4.106, 4.848, 5.689,
               6.682, 7.626, 9.16,  10.66, 11.841, 13.098, 15.451,
               17.094, 19.423, 21.682, 23.342, 24.920, 28.174, 30.736,
               33.313, 36.109, 39.673, 41.955, 45.056, 48.647, 50.983,
               55.514, 59.278, 61.311, 64.016)
    
    lame <- NA
    lamm <- NA
    lamq <- NA
    
    # Median-unbiased estimator of lambda_g for given values of the test
    # statistics are obtained using the procedure described in the 
    # footnote to Stock and Watson (1998) Table 3.
    if (ew <= valew[1]) {
        lame <- 0
    } else {
        for (i in 1:(length(valew)-1)) {
            if ((ew > valew[i]) & (ew <= valew[i+1])) {
                lame <- i-1+(ew-valew[i])/(valew[i+1]-valew[i])
            }
        }
    }

    if (mw <= valmw[1]) {
        lamm <- 0
    } else {
        for (i in 1:(length(valmw)-1)) {
            if ((mw > valmw[i]) & (mw <= valmw[i+1])) {
                lamm <- i-1+(mw-valmw[i])/(valmw[i+1]-valmw[i])
            }
        }
    }

    if (qlr <= valql[1]) {
        lamq <- 0
    } else {
        for (i in 1:(length(valql)-1)) {
            if ((qlr > valql[i]) & (qlr <= valql[i+1])) {
                lamq <- i-1+(qlr-valql[i])/(valql[i+1]-valql[i])
            }
        }
    }
    if (is.na(lame) | is.na(lamm) | is.na(lamq)) {
            print("At least one statistic has an NA value. 
                   Check to see if your EW, MW, and/or QLR value is outside of Table 3.")
    }
    
    stats <- c(ew,mw,qlr)
    lams  <- c(lame,lamm,lamq)
    return(lame/T)
}

### Kalman Filter Programs

### *kalman.states.R*
The function kalman.states() calls the functions kalman.states.filtered() and kalman.states.smoothed() to apply the Kalman filter and smoother. It takes as input the coefficient matrices for the given state-space model as well as the conditional expectation and covariance matrix of the initial state, xi.tm1tm1 $\left( \xi_{t-1|t-1} \right)$ and P.tm1tm1 $ \left( P_{t-1|t-1} \right)$ respectively.

In [ ]:
kalman.states <- function(xi.tm1tm1, P.tm1tm1, F, Q, A, H, R, y, x) {
  filtered <- kalman.states.filtered(xi.tm1tm1, P.tm1tm1, F, Q, A, H, R, y, x)
  smoothed <- kalman.states.smoothed(filtered$xi.ttm1, filtered$P.ttm1, filtered$xi.tt, filtered$P.tt,
                                     F, Q, A, H, R, y, x)
  return(list("filtered"=filtered, "smoothed"=smoothed))
}
kalman.states.filtered <- function(xi.tm1tm1, P.tm1tm1, F, Q, A, H, R, y, x, t=1) {
  xi.ttm1 <- as.vector(F %*% xi.tm1tm1)
  P.ttm1 <- F %*% P.tm1tm1 %*% t(F) + Q
  prediction.error <- (as.vector(y[t,]) - as.vector(t(A) %*% as.vector(x[t,])) - as.vector(t(H) %*% xi.ttm1))
  HPHR <- t(H) %*% P.ttm1 %*% H + R
  xi.tt <- xi.ttm1 + as.vector(P.ttm1 %*% H %*% solve(HPHR, prediction.error))
  P.tt <- P.ttm1 - P.ttm1 %*% H %*% solve(HPHR, t(H) %*% P.ttm1)
  if (t == dim(y)[1]) {
      return(list("xi.ttm1"=xi.ttm1, "P.ttm1"=P.ttm1, "xi.tt"=xi.tt, "P.tt"=P.tt))
  } else {
      tmp <- kalman.states.filtered(xi.tt, P.tt, F, Q, A, H, R, y, x, t+1)
      return(list("xi.ttm1"=rbind(xi.ttm1, tmp$xi.ttm1),
                  "P.ttm1"=rbind(P.ttm1, tmp$P.ttm1),
                  "xi.tt"=rbind(xi.tt, tmp$xi.tt),
                  "P.tt"=rbind(P.tt, tmp$P.tt)))
  }
}
kalman.states.smoothed <- function(xi.ttm1.array, P.ttm1.array, xi.tt.array, P.tt.array,
                                   F, Q, A, H, R, y, x, t=dim(y)[1], xi.tp1T=NA, P.tp1T=NA) {
  n <- dim(xi.ttm1.array)[2]
  if (t == dim(y)[1]) {
    xi.tT <- xi.tt.array[t,]
    P.tT <- P.tt.array[((t-1)*n+1):(t*n),]
    tmp <- kalman.states.smoothed(xi.ttm1.array, P.ttm1.array, xi.tt.array, P.tt.array,
                                  F, Q, A, H, R, y, x, t-1, xi.tT, P.tT)
    return(list("xi.tT"=rbind(tmp$xi.tT, xi.tT),
                "P.tT" =rbind(tmp$P.tT, P.tT)))
  } else {
    P.tt <- P.tt.array[((t-1)*n+1):(t*n),]
    P.tp1t <- P.ttm1.array[(t*n+1):((t+1)*n),]
    J.t <- P.tt %*% t(F) %*% solve(P.tp1t)
    xi.tt <- xi.tt.array[t,]
    xi.tp1t <- xi.ttm1.array[t+1,]
    xi.tT <- xi.tt + as.vector(J.t %*% (xi.tp1T - xi.tp1t))
    P.tT <- P.tt + J.t %*% (P.tp1T - P.tp1t) %*% t(J.t)
    if (t > 1) {
      tmp <- kalman.states.smoothed(xi.ttm1.array, P.ttm1.array, xi.tt.array, P.tt.array,
                                    F, Q, A, H, R, y, x, t-1, xi.tT, P.tT)
      return(list("xi.tT"=rbind(tmp$xi.tT, xi.tT),
                  "P.tT" =rbind(tmp$P.tT, P.tT)))
    } else {
      return(list("xi.tT"=xi.tT, "P.tT"=P.tT))
    }
  }
}

### *kalman.states.wrapper.R*
This is a wrapper function for *kalman.states.R* that specifies inputs based on the estimation stage.

In [ ]:
kalman.states.wrapper <- function(parameters, y.data, x.data, stage = NA,
                                  lambda.g=NA, lambda.z=NA, xi.00=NA, P.00=NA){

    if (stage == 1) {
        out <- unpack.parameters.stage1(parameters, y.data, x.data,
                                        xi.00, P.00)
    } else if (stage == 2) {
        out <- unpack.parameters.stage2(parameters, y.data, x.data,
                                        lambda.g, xi.00, P.00)
    } else if (stage == 3) {
        out <- unpack.parameters.stage3(parameters, y.data, x.data,
                                        lambda.g, lambda.z, xi.00, P.00)
    } else {
        stop('You need to enter a stage number in kalman.states.wrapper.')
    }

  for (n in names(out)) {
      eval(parse(text=paste0(n, "<-out$", n)))
  }
  T <- dim(y.data)[1]
  states <- kalman.states(xi.00, P.00, F, Q, A, H, R, y.data, x.data)
  if (stage == 1) {
      states$filtered$xi.tt <- states$filtered$xi.tt + cbind(1:T,0:(T-1),-1:(T-2)) * parameters[5]
      states$smoothed$xi.tT <- states$smoothed$xi.tT + cbind(1:T,0:(T-1),-1:(T-2)) * parameters[5]
  }
return(states)}

### Log Likelihood Programs

### *kalman.log.likelihood.R*
This function takes as input the coefficient matrices of the given state-space model and the conditional expectation and covariance matrix of the initial state and returns the log likelihood value and a vector with the log likelihood at each time $t$.

In [ ]:
kalman.log.likelihood <- function(xi.tm1tm1, P.tm1tm1, F, Q, A, H, R, y, x) {
    T <- dim(y)[1]
    n <- dim(y)[2]
    ll.vec <- matrix(NA,T,1)
    ll.cum <- 0
    xi.tt <- xi.tm1tm1
    P.tt  <- P.tm1tm1    
    for (t in 1:T){

        xi.ttm1 <- F %*% xi.tt
        P.ttm1 <- F %*% P.tt %*% t(F) + Q
        prediction.error <- (as.vector(y[t,]) - as.vector(t(A) %*% as.vector(x[t,])) - as.vector(t(H) %*% xi.ttm1))
        HPHR <- t(H) %*% P.ttm1 %*% H + R
        ll.vec[t] <- drop(-(n / 2) * log(2 * atan(1) * 4) - 0.5 * log(det(HPHR))
                          -0.5 * prediction.error %*% solve(HPHR, prediction.error))
        ll.cum <- ll.cum + ll.vec[t]

        xi.tt <- xi.ttm1 + P.ttm1 %*% H %*% solve(HPHR, prediction.error)
        P.tt  <- P.ttm1 - P.ttm1 %*% H %*% solve(HPHR, t(H) %*% P.ttm1)
    }
    return(list("ll.vec"=ll.vec,"ll.cum"=ll.cum))
}

### *log.likelihood.wrapper.R*
This is a wrapper function for *kalman.log.likelihood.R* that specifies inputs based on the estimation stage.

In [ ]:
log.likelihood.wrapper <- function(parameters, y.data, x.data, stage = NA,
                                   lambda.g=NA, lambda.z=NA, xi.00=NA, P.00=NA){

    if (stage == 1) {
        out <- unpack.parameters.stage1(parameters, y.data, x.data,
                                        xi.00, P.00)
    } else if (stage == 2) {
        out <- unpack.parameters.stage2(parameters, y.data, x.data,
                                        lambda.g, xi.00, P.00)
    } else if (stage == 3) {
        out <- unpack.parameters.stage3(parameters, y.data, x.data,
                                        lambda.g, lambda.z, xi.00, P.00)
    } else {
        stop('You need to enter a stage number in log.likelihood.wrapper.')
    }


  for (n in names(out)) {
      eval(parse(text=paste0(n, "<-out$", n)))
  }
  return(kalman.log.likelihood(xi.00, P.00, F, Q, A, H, R, y.data, x.data))
}

### *kalman.standard.errors.R*
This function computes confidence intervals and corresponding standard errors for the estimates of the states using Hamilton's (1986) Monte Carlo procedure that accounts for both filter and parameter uncertainty. See footnote 7 in HLW.

In [ ]:
kalman.standard.errors <- function(T, states, theta, y.data, x.data, stage,
                                   lambda.g, lambda.z, xi.00, P.00, niter = 5000,
                                   a3.constraint=NA, b2.constraint=NA) {

    print('Computing Standard Errors')
    
    # Set a3.constraint to -0.0025 if a constraint is not specified in stage 3
    if (is.na(a3.constraint)) {
        a3.constraint <- -0.0025
    }
    # Set b2.constraint to 0.025 if a constraint is not specified in stage 3
    if (is.na(b2.constraint)) {
        b2.constraint <- 0.025
    }

    print("Standard Error Procedure: a3.constraint")
    print(a3.constraint)
    
    print("Standard Error Procedure: b2.constraint")
    print(b2.constraint)

    n.params <- length(theta)
    n.state.vars <- length(xi.00)

    # Return vector of log likelihood values at each time t 
    log.likelihood.estimated.vector <- log.likelihood.wrapper(theta, y.data, x.data, stage = 3,
                                                              lambda.g=lambda.g, lambda.z=lambda.z, 
                                                              xi.00=xi.00, P.00=P.00)$ll.vec
    stin <- states$smoothed$xi.tT[1,] # First smoothed state vector
    pp1  <- states$filtered$P.ttm1[1:n.state.vars,] # First covariance matrix
    eigenstuff.pp1   <- eigen(pp1)
    eigenvectors.pp1 <- eigenstuff.pp1$vectors # Eigenvectors of first covariance matrix
    # Eigenvectors without a positive first entry are multiplied by -1 to ensure
    # consistency across different versions of R, which choose the sign differently
    for (l in 1:n.state.vars) {
        if (eigenvectors.pp1[1,l] < 0 ) { eigenvectors.pp1[,l] <- -eigenvectors.pp1[,l] }
    } 
    eigenvalues.pp1  <- eigenstuff.pp1$value   # Eigenvalues of first covariance matrix
    dg   <- diag(x = eigenvalues.pp1) 
    hh2  <- eigenvectors.pp1 %*% sqrt(dg)    

    # Compute information matrix from difference in gradients of the likelihood function
    # from varying theta (parameter vector) values
    likelihood.gradient <- matrix(NA,T,n.params)
    for (i in 1:n.params){
        delta   <- max(theta[i]*1e-6, 1e-6)
        d.theta <- theta
        d.theta[i] <- theta[i] + delta
        likelihood.gradient[,i] <-  (log.likelihood.wrapper(d.theta, y.data, x.data, stage = 3,
                                                            lambda.g=lambda.g, lambda.z=lambda.z, 
                                                            xi.00=xi.00, P.00=P.00)$ll.vec - 
                                         log.likelihood.estimated.vector)/delta
    }
    info <- solve(t(likelihood.gradient) %*% likelihood.gradient) # Information matrix
    bse <- sqrt(diag(info))
    t.stats <- abs(theta) / bse

    # Smoothed estimates
    g      <- 4 * states$smoothed$xi.tT[,4]
    ypot   <- states$smoothed$xi.tT[,1]
    z      <- states$smoothed$xi.tT[,6]
    rstar  <- g + z

    # cum1 cumulates terms for parameter uncertainty;
    # cum2 cumulates terms for filter uncertainty
    cum1 <- matrix(0,T,3)
    cum2 <- matrix(0,T,3)
    eigenstuff.info   <- eigen(info)
    eigenvectors.info <- eigenstuff.info$vectors # Eigenvectors of information matrix
    # Eigenvectors without a positive first entry are multiplied by -1 to ensure
    # consistency across different versions of R, which choose the sign differently
    for (l in 1:n.params) {
        if (eigenvectors.info[1,l] < 0 ) { eigenvectors.info[,l] <- -eigenvectors.info[,l] }
    }
    eigenvalues.info  <- eigenstuff.info$value # Eigenvalues of information matrix
    dg <- diag(x = eigenvalues.info)
    hh <- eigenvectors.info %*% sqrt(dg)

    set.seed(50)

    # Store the number of draws excluded for violating constraints
    good.draws                 <- 0
    excluded.draw.counter      <- 0
    excluded.draw.counter.a3   <- 0
    excluded.draw.counter.b2   <- 0
    excluded.draw.counter.a1a2 <- 0

    # See HLW footnote 7 for description of procedure
    # niter is the number of iterations; we discard draws that violate constraints
    while (good.draws < niter) {
      theta.i <- (hh %*% rnorm(n.params) + theta)[,1] 
      if ( (theta.i[3] <= a3.constraint) & (theta.i[5] >= b2.constraint) & (theta.i[1] + theta.i[2] < 1) ) {
          xi.00.i  <- c(t(hh2 %*% rnorm(n.state.vars) + stin)) 
          states.i <- kalman.states.wrapper(theta.i, y.data, x.data, stage, lambda.g, lambda.z, xi.00.i, pp1)

          g.i    <- 4 * states.i$smoothed$xi.tT[,4]
          ypot.i <- states.i$smoothed$xi.tT[,1]
          z.i    <- states.i$smoothed$xi.tT[,6]
          r.i    <- g.i + z.i          
                    
          cum1[,1] <- cum1[,1]+(ypot.i-ypot)^2
          cum1[,2] <- cum1[,2]+(r.i-rstar)^2
          cum1[,3] <- cum1[,3]+(g.i-g)^2

          P.ttm1.i   <- states.i$smoothed$P.tT
          P.ttm1.i.f <- states.i$filtered$P.tt
          for (j in 1:(T-1)){
              cum2[j,1]  <- cum2[j,1] + P.ttm1.i[(j * n.state.vars +1),1]
              cum2[j,2]  <- cum2[j,2] + 16 * P.ttm1.i[(j*n.state.vars+4),4] + P.ttm1.i[(j*n.state.vars+6),6]
              cum2[j,3]  <- cum2[j,3] + P.ttm1.i[(j*n.state.vars+4),4]
          }
          cum2[T,1] <- cum2[T,1] + P.ttm1.i.f[((T-1)*n.state.vars+1),1]
          cum2[T,2] <- cum2[T,2] + (16 * P.ttm1.i.f[((T-1)*n.state.vars+4),4] + P.ttm1.i.f[((T-1)*n.state.vars+6),6])
          cum2[T,3] <- cum2[T,3] + P.ttm1.i.f[((T-1)*n.state.vars+4),4]
          good.draws <- good.draws + 1
      } else {
          excluded.draw.counter <- excluded.draw.counter + 1
          if (theta.i[3] > a3.constraint) {
              excluded.draw.counter.a3 <- excluded.draw.counter.a3 + 1
          }
          if (theta.i[5] < b2.constraint) {
              excluded.draw.counter.b2 <- excluded.draw.counter.b2 + 1
          }
          if ((theta.i[1] + theta.i[2]) >= 1) {
              excluded.draw.counter.a1a2 <- excluded.draw.counter.a1a2 + 1
          }
      }
    }
    cum1 <- cum1/niter # Measure of parameter uncertainty
    cum2 <- cum2/niter # Measure of filter uncertainty
    cum2[,3] <- 16*cum2[,3] # Variance for growth at an annualized rate

    # Standard errors for estimates of the states
    # Order: y*, r*, g
    se <- sqrt(cum1 + cum2)

    rm(.Random.seed)
    return(list("se.mean"=colMeans(se),
                "se"=se,"t.stats"=t.stats,"bse"=bse,
                "number.excluded"=excluded.draw.counter,
                "number.excluded.a3"=excluded.draw.counter.a3,
                "number.excluded.b2"=excluded.draw.counter.b2,
                "number.excluded.a1a2"=excluded.draw.counter.a1a2))
}    

### *calculate.covariance.R*
This function calculates the covariance matrix of the initial state from the gradients of the likelihood function.

In [ ]:
calculate.covariance <- function(initial.parameters,theta.lb,theta.ub,y.data,x.data,
                                 stage,lambda.g=NA,lambda.z=NA,xi.00) {

  n.state.vars <- length(xi.00)

  # Set covariance matrix equal to 0.2 times the identity matrix
  P.00 <- diag(0.2,n.state.vars,n.state.vars)
  
  # Get parameter estimates via maximum likelihood
  f <- function(theta) {return(-log.likelihood.wrapper(theta, y.data, x.data, stage, lambda.g, lambda.z, xi.00, P.00)$ll.cum)}
  nloptr.out <- nloptr(initial.parameters, f, eval_grad_f=function(x) {gradient(f, x)},
                       lb=theta.lb,ub=theta.ub,opts=list("algorithm"="NLOPT_LD_LBFGS","xtol_rel"=1.0e-8))
  theta <- nloptr.out$solution
  
  # Run Kalman filter with above covariance matrix and corresponding parameter estimates
  states <- kalman.states.wrapper(theta, y.data, x.data, stage, lambda.g, lambda.z, xi.00, P.00)

  # Save initial covariance matrix 
  P.00 <- states$filtered$P.ttm1[1:n.state.vars,]
  
return(P.00)}

### *format.output.R*
This function generates a dataframe to be written to a CSV containing one-sided estimates, parameter values, standard errors, and other statistics of interest.

In [ ]:
format.output <- function(country.estimation, one.sided.est.country, real.rate.country, start, end, run.se = TRUE) {
    output.country <- data.frame(matrix(NA,dim(one.sided.est.country)[1],22))
    
    output.country[,1]   <- seq(from = (as.Date(ti(shiftQuarter(start,-1),'quarterly'))+1), 
                                to = (as.Date(ti(shiftQuarter(end,-1),tif='quarterly'))+1), by = 'quarter')
    output.country[,2:5] <- one.sided.est.country

    output.country[1,7]    <- "Parameter Point Estimates"
    output.country[2,7:15] <- c("a_1","a_2","a_3","b_1","b_2","sigma_1","sigma_2","sigma_4","a_1 + a_2")
    output.country[3,7:14] <- country.estimation$out.stage3$theta
    output.country[3,15]   <- country.estimation$out.stage3$theta[1] + country.estimation$out.stage3$theta[2]
    # Include standard errors in output only if run.se switch is TRUE
    if (run.se) {
        output.country[4,7]    <- "T Statistics"
        output.country[5,7:14] <- country.estimation$out.stage3$se$t.stats
        
        output.country[8,7]    <- "Average Standard Errors"
        output.country[9,7:9]  <- c("y*","r*","g")
        output.country[10,7:9] <- country.estimation$out.stage3$se$se.mean
        
        output.country[12,7]   <- "Restrictions on MC draws: a_3 < -0.0025; b_2 > 0.025; a_1 + a_2 < 1"
        output.country[13,7]   <- "Draws excluded:"
        output.country[13,9]   <- country.estimation$out.stage3$se$number.excluded
        output.country[13,10]  <- "Total:"; output.country[13,11] <- niter
        output.country[14,7]   <- "Percent excluded:"
        output.country[14,9]   <- as.numeric(output.country[13,9]) / 
                                 (as.numeric(output.country[13,9]) + as.numeric(output.country[13,11]))
        output.country[15,7]   <- "Draws excluded because a_3 > -0.0025:"
        output.country[15,11]  <- country.estimation$out.stage3$se$number.excluded.a3
        output.country[16,7]   <- "Draws excluded because b_2 <  0.025:"
        output.country[16,11]  <- country.estimation$out.stage3$se$number.excluded.b2
        output.country[17,7]   <- "Draws excluded because a_1 + a_2 < 1:"
        output.country[17,11]  <- country.estimation$out.stage3$se$number.excluded.a1a2
    }
    
    output.country[19,7]  <- "Signal-to-noise Ratios"
    output.country[20,7]  <- "lambda_g"; output.country[20,8] <- country.estimation$lambda.g
    output.country[21,7]  <- "lambda_z"; output.country[21,8] <- country.estimation$lambda.z
    output.country[19,11] <- "Log Likelihood"; output.country[20,11] <- country.estimation$out.stage3$log.likelihood

    output.country[24,7]    <- "State vector: [y_{t}* y_{t-1}* y_{t-2}* g_{t-1} g_{t-2} z_{t-1} z_{t-2}]"
    output.country[25,7]    <- "Initial State Vector"
    output.country[26,7:13] <- country.estimation$out.stage3$xi.00
    output.country[28,7]    <- "Initial Covariance Matrix"
    output.country[29:35,7:13] <- country.estimation$out.stage3$P.00

    if (run.se) {
        output.country[,17]    <- seq(from = (as.Date(ti(shiftQuarter(start,-1),'quarterly'))+1), 
                                      to = (as.Date(ti(shiftQuarter(end,-1),tif='quarterly'))+1), by = 'quarter')
        output.country[,18:20] <- country.estimation$out.stage3$se$se
    }
    
    output.country[,22] <- real.rate.country[5:length(real.rate.country)] - 
                            country.estimation$out.stage3$rstar.filtered

return(output.country)}

## Run Estimation for Each Economy

### United States

In [ ]:
# Read in output of prepare.rstar.data.us.R
us.data <- read.table("inputData/rstar.data.us.csv",
                      sep = ',', na.strings = ".", header=TRUE, stringsAsFactors=FALSE)

us.log.output             <- us.data$gdp.log
us.inflation              <- us.data$inflation
us.inflation.expectations <- us.data$inflation.expectations
us.nominal.interest.rate  <- us.data$interest
us.real.interest.rate     <- us.nominal.interest.rate - us.inflation.expectations

# Run HLW estimation for the US
us.estimation <- run.hlw.estimation(us.log.output, us.inflation, us.real.interest.rate, us.nominal.interest.rate,
                                    a3.constraint = a3.constraint, b2.constraint = b2.constraint, run.se = run.se)

# One-sided (filtered) estimates
one.sided.est.us <- cbind(us.estimation$out.stage3$rstar.filtered,
                          us.estimation$out.stage3$trend.filtered,
                          us.estimation$out.stage3$z.filtered,
                          us.estimation$out.stage3$output.gap.filtered)

# Save one-sided estimates to CSV
write.table(one.sided.est.us, 'output/one.sided.est.us.csv', row.names = FALSE, 
            col.names = c("rstar","g","z","output gap"), quote = FALSE, sep = ',', na = ".")

# Save output to CSV
output.us <- format.output(us.estimation, one.sided.est.us, us.real.interest.rate, sample.start, sample.end)
write.table(output.us, 'output/output.us.csv', col.names = output.col.names, 
            quote=FALSE, row.names=FALSE, sep = ',', na = '')

### Canada

In [ ]:
# Read in output of prepare.rstar.data.ca.R
ca.data <- read.table("inputData/rstar.data.ca.csv",
                      sep = ',', na.strings = ".", header=TRUE, stringsAsFactors=FALSE)

ca.log.output             <- ca.data$gdp.log
ca.inflation              <- ca.data$inflation
ca.inflation.expectations <- ca.data$inflation.expectations
ca.nominal.interest.rate  <- ca.data$interest
ca.real.interest.rate     <- ca.nominal.interest.rate - ca.inflation.expectations

# Run HLW estimation for Canada
ca.estimation <- run.hlw.estimation(ca.log.output, ca.inflation, ca.real.interest.rate, ca.nominal.interest.rate,
                                    a3.constraint = a3.constraint, b2.constraint = b2.constraint, run.se = run.se)

# One-sided (filtered) estimates
one.sided.est.ca <- cbind(ca.estimation$out.stage3$rstar.filtered,
                          ca.estimation$out.stage3$trend.filtered,
                          ca.estimation$out.stage3$z.filtered,
                          ca.estimation$out.stage3$output.gap.filtered)

# Save one-sided estimates to CSV
write.table(one.sided.est.ca, 'output/one.sided.est.ca.csv', row.names = FALSE, 
            col.names = c("rstar","g","z","output gap"), quote = FALSE, sep = ',', na = ".")

# Save output to CSV
output.ca <- format.output(ca.estimation, one.sided.est.ca, ca.real.interest.rate, sample.start, sample.end, run.se = run.se)
write.table(output.ca, 'output/output.ca.csv', col.names = output.col.names, 
            quote=FALSE, row.names=FALSE, sep = ',', na = '')

### Euro Area

In [ ]:
# Read in output of prepare.rstar.data.ea.R
ea.data <- read.table("inputData/rstar.data.ea.csv",
                      sep = ',', na.strings = ".", header=TRUE, stringsAsFactors=FALSE)

ea.log.output             <- ea.data$gdp.log
ea.inflation              <- ea.data$inflation
ea.inflation.expectations <- ea.data$inflation.expectations
ea.nominal.interest.rate  <- ea.data$interest
ea.real.interest.rate     <- ea.nominal.interest.rate - ea.inflation.expectations

# Run HLW estimation for the Euro Area
ea.estimation <- run.hlw.estimation(ea.log.output, ea.inflation, ea.real.interest.rate, ea.nominal.interest.rate,
                                    a3.constraint = a3.constraint, b2.constraint = b2.constraint, run.se = run.se)

# One-sided (filtered) estimates
one.sided.est.ea <- cbind(ea.estimation$out.stage3$rstar.filtered,
                          ea.estimation$out.stage3$trend.filtered,
                          ea.estimation$out.stage3$z.filtered,
                          ea.estimation$out.stage3$output.gap.filtered)

# Save one-sided estimates to CSV
write.table(one.sided.est.ea, 'output/one.sided.est.ea.csv', row.names = FALSE, 
           col.names = c("rstar","g","z","output gap"), quote = FALSE, sep = ',', na = ".")

# Save output to CSV
output.ea <- format.output(ea.estimation, one.sided.est.ea, ea.real.interest.rate, ea.sample.start, 
                           sample.end, run.se = run.se)
write.table(output.ea, 'output/output.ea.csv', col.names = output.col.names, 
            quote=FALSE, row.names=FALSE, sep = ',', na = '')

### United Kingdom

In [ ]:
# Read in output of prepare.rstar.data.uk.R
uk.data <- read.table("inputData/rstar.data.uk.csv",
                      sep = ',', na.strings = ".", header=TRUE, stringsAsFactors=FALSE)


uk.log.output             <- uk.data$gdp.log
uk.inflation              <- uk.data$inflation
uk.inflation.expectations <- uk.data$inflation.expectations
uk.nominal.interest.rate  <- uk.data$interest
uk.real.interest.rate     <- uk.nominal.interest.rate - uk.inflation.expectations

# Run HLW estimation for the UK
uk.estimation <- run.hlw.estimation(uk.log.output, uk.inflation, uk.real.interest.rate, uk.nominal.interest.rate,
                                    a3.constraint = a3.constraint, b2.constraint = b2.constraint, run.se = run.se)

# One-sided (filtered) estimates
one.sided.est.uk <- cbind(uk.estimation$out.stage3$rstar.filtered,
                          uk.estimation$out.stage3$trend.filtered,
                          uk.estimation$out.stage3$z.filtered,
                          uk.estimation$out.stage3$output.gap.filtered)

# Save one-sided estimates to CSV
write.table(one.sided.est.uk, 'output/one.sided.est.uk.csv', row.names = FALSE, 
            col.names = c("rstar","g","z","output gap"), quote = FALSE, sep = ',', na = ".")

# Save output to CSV
output.uk <- format.output(uk.estimation, one.sided.est.uk, uk.real.interest.rate, sample.start, sample.end, run.se = run.se)
write.table(output.uk, 'output/output.uk.csv', col.names = output.col.names, 
            quote=FALSE, row.names=FALSE, sep = ',', na = '')